### Librairy Import

In [2]:
#!pip install virtualenv

In [3]:
#!virtualenv -p python3.10 /content/virtualenvforIA

In [4]:
#!source /content/virtualenvforIA/bin/activate


In [1]:
!pip install pdfplumber
!pip install transformers
!pip install datasets
!pip install evaluate
#!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not curr

In [5]:
#!pip install functools

In [2]:
#Cas dossier personnel
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### JSON DATASET BUILDER

In [3]:
# JSON Dataset builder - Main
import os
import pathlib
import json
import pdfplumber
import re
from pprint import pprint
#import pandas as pd
#import numpy as np
from tqdm import tqdm

class Classifier_Attestation_Propriete:
  '''Convertit un PDF en fichier json classifiant les données selon leurs sequences et tokens associés pour générer des données d'entrainement'''

  def __init__(self, path_pdf):
    # Instanciation des attributs de l'objet PDFToJsonDataset
    self.path_pdf = path_pdf
    self.flatten_text = ""
    self.texts = []
    self.lines = []
    self.num_pages = 0
    self.token_visited = []



    self.json_dataset = {} # Contient tous les attributs extraits du pdf mise en forme

    #for line in self.lines[0]:
    #  print(line)

  # CONSTRUCT THE JSON DATASET FOR SEQUENCE AND TOKEN CLASSIFICATION

  def import_pdf(self):
    '''Import le PDF en mémoire'''
    # Importation PDF et Conversion Texte Brut
    with pdfplumber.open(self.path_pdf) as pdf:
      pages = pdf.pages
      self.num_pages = len(pages)
      self.texts = [pages[i].extract_text() for i in range(self.num_pages)]
      self.lines = [self.texts[i].split("\num_match") for i in range(self.num_pages)]

    self.set_flatten_text()
    return

  def extract(self):
    '''Renvoie en format JSON le Dataset extrait du pdf en format pdf pour ensuite pouvoir être entrainé par l'IA'''

    self.set_flatten_text()
    if len(self.flatten_text) == 0:
       return {}

    self.json_dataset["Sections"] = self.get_json_sections()
    self.json_dataset["Notaire"] = self.get_json_notaire_1()
    self.json_dataset["Qualification Juridique"] = self.get_json_qualif_juridique_2()
    self.json_dataset["Etat Civil Décès"] = self.get_json_etat_civil_D_3()
    self.json_dataset["Etat Civil Heritier"] = self.get_json_etat_civil_H_3()
    self.json_dataset["Designation"] = self.get_json_designation_4()
    self.json_dataset["Evaluation"] = self.get_json_evaluation_5()
    self.json_dataset["Etat Descriptif de Division"] = self.get_json_etat_descriptif_6()
    self.json_dataset["Effet Relatif"] = self.get_json_effet_relatif_7()
    self.json_dataset["Droits Transmis"] = self.get_json_droits_transmis_8()
    self.json_dataset["Requisition Publication"] = self.get_json_requisition_publication_9()
    self.json_dataset["Certification Attestation"] = self.get_json_certification_attestation_10()
    self.json_dataset["Certification Identite"] = self.get_json_certification_identite_11()
    self.json_dataset["Contexte"] = self.get_json_context()

    return self.json_dataset

  # Useful Functions

  def set_flatten_text(self):
    '''Réinitialise la valeur du texte aplati entre un String de tout le pdf'''
    self.flatten_text = ""
    for i in range(self.num_pages):
      self.flatten_text = self.flatten_text + self.texts[i]
    return

  def update_flatten_text(self, first_quote = "", last_quote = "", index_before = 0):
    '''Actualise le text aplati à partir d'une citation'''
    self.set_flatten_text()

    if len(first_quote) >= 1:
        text_split = re.split(first_quote, self.flatten_text, maxsplit=1)
        try:
            self.flatten_text = text_split[1]
        except:
            self.flatten_text = ""
    else:
        self.flatten_text = ""
    if len(last_quote) >= 1:
        text_split_zoom = re.split(last_quote, self.flatten_text, maxsplit=1)
        self.flatten_text = text_split_zoom[0]

    if len(first_quote) == 0:
        last_index = index_before
    else:
        last_index = len(text_split[0].replace("\n", " "))
    return last_index

  def get_match(self, pattern, index_before = 0, n_groups = 0, replace = ""):
    '''Renvoie la valeur de l'expression extraite et sa position en ligne si celui-ci existe'''

    # EXPLICATION :
    # re = REGULAR EXPRESSION ou REGEX
    # pattern est un exemple d'Expression Régulière
    # re.search(pattern, self.flatten_text) effectue un recherche d'expression régulière selon le language REGEX (REGular EXpression) pour détecter les expressions à extraire
    # flags=re.DOTALL précise qu'il ne doit pas prendre en compte les sauts de ligne
    # re.sub(r'[\n,.]', ' ', text) pour remplacer les sauts de lignes, les virgules et les points par des espaces
    # match.group(n_groups) renvoie le groupe extrait n° n_groups --> 0 == Tout / 1 == Juste la partie entre 1ère parenthèse du REGEX / 2 == Dans la deuxième parenthèse / etc
    # text.strip() supprimer les espaces inutiles
    # match.start() est l'indice du caractère au début de l'expression extraite
    # len(text[:match.start()].splitlines()) compte le nombre de ligne avant cette expression extraite

    match = re.search(pattern, self.flatten_text, flags=re.DOTALL)
    if match:
      expression = re.sub(r'[\n]', ' ', match.group(n_groups)).strip()

      index_position = [index_before + match.start(n_groups) + 1]# Verifier si les indices correspondent
      n = len(expression)
      for word in expression.split(" "):
          index_position.append(index_position[-1] + len(word) + 1)
      index_position = index_position[:-1]

      # end_index = index_before + match.end(n_groups) # + len(self.flatten_text[:match.start()+1].splitlines())
      if(len(replace)>=1):
        expression = expression.replace(replace, " ").strip()
      return [[expression], [index_position]]
    else:
      #print("No Data")
      return [[], []]

  def get_matches(self, pattern, index_before = 0, n_groups = 0, replace = "", selected_groups=[]):
    '''Renvoie la valeur de l'expression extraite et sa position en ligne si celui-ci existe'''

    # EXPLICATION :
    # re = REGULAR EXPRESSION ou REGEX
    # pattern est un exemple d'Expression Régulière
    # re.finditer(pattern, self.flatten_text) effectue un recherche d'expression régulière selon le language REGEX (REGular EXpression) pour détecter les expressions à extraire
    # flags=re.DOTALL précise qu'il ne doit pas prendre en compte les sauts de ligne
    # re.sub(r'[\n,.]', ' ', text) pour remplacer les sauts de lignes, les virgules et les points par des espaces
    # match.group(n_groups) renvoie le groupe extrait n° n_groups --> 0 == Tout / 1 == Juste la partie entre 1ère parenthèse du REGEX / 2 == Dans la deuxième parenthèse / etc
    # text.strip() supprimer les espaces inutiles
    # match.start() est l'indice du caractère au début de l'expression extraite
    # len(text[:match.start()].splitlines()) compte le nombre de ligne avant cette expression extraite

    matches_iter = re.finditer(pattern, self.flatten_text, flags=re.DOTALL)
    matches, positions = [], []

    for match in matches_iter:

        expression = ""
        if len(selected_groups) == 0:
            expression = expression + re.sub(r"[\n]", " ", match.group(n_groups)).strip()
            if(len(replace)>=1):
                expression = expression.replace(replace, " ").strip()

        for n_groups in selected_groups:
            expression = expression + re.sub(r"[\n]", " ", match.group(n_groups)).strip()
            if(len(replace)>=1):
                expression = expression.replace(replace, " ").strip()

        index_position = [index_before + match.start(n_groups) + 1] # Verifier si les indices correspondent
        n = len(expression)
        for word in expression.split(" "):
            index_position.append(index_position[-1] + len(word) + 1)
        index_position = index_position[:-1]

        matches.append(expression)
        positions.append(index_position)

    return [matches, positions]

  # GET JSON DATASET

  def get_json_sections(self):
    '''Renvoie le JSON Dataset des différentes Clauses de l'acte'''
    self.set_flatten_text()

    # Ces attributs décrivent la structure des multiples parties indépendantes à analyser d'un acte d'une attestation de propriété
    named_label = "Sections"
    subdataset = {}
    subdataset["Texte_Applicable_S"] = self.set_normed_data_default(self.get_match(r"TEXT[E|ES]\sAPPLICABLE", n_groups=0), named_label)
    subdataset["Personne_Decedee_S"] = self.set_normed_data_default(self.get_match(r"(PERSONN[E|ES]\sD[E|É]C[E|É]D[E|É][E|ES])", n_groups=0), named_label)
    subdataset["Devolution_Successorale_S"] = self.set_normed_data_default(self.get_match(r"D[E|É]VOLUTION\sSUCCESSORALE", n_groups=0), named_label)
    subdataset["Qualities_Hereditaires_S"] = self.set_normed_data_default(self.get_match(r"QUALIT[E|ES]\sH[E|É]R[E|É]DITAIR[E|ES]", n_groups=0), named_label)
    subdataset["Visa_des_Actes_S"] = self.set_normed_data_default(self.get_match(r"VISA\sDES\sACTES", n_groups=0), named_label)
    subdataset["Acceptation_de_la_Succession_S"] = self.set_normed_data_default(self.get_match(r"ACCEPTATION\sDE\sLA\sSUCCESSION", n_groups=0), named_label)
    subdataset["Designation_S"] = self.set_normed_data_default(self.get_match(r"D[E|É]SIGNATION", n_groups=0), named_label)
    subdataset["Lotissement_S"] = self.set_normed_data_default(self.get_match(r"Lotissement", n_groups=0), named_label)
    subdataset["Evaluation_S"] = self.set_normed_data_default(self.get_match(r"[E|É]VALUATION", n_groups=0), named_label)
    subdataset["Effet_Relatif_S"] = self.set_normed_data_default(self.get_match(r"EFFET\sRELATIF", n_groups=0), named_label)
    subdataset["Origine_de_Propriete_S"] = self.set_normed_data_default(self.get_match(r"ORIGINE\sDE\sPROPRI[E|É]T[E|É]", n_groups=0), named_label)
    subdataset["Servitude_S"] = self.set_normed_data_default(self.get_match(r"SERVITUDES", n_groups=0), named_label)
    subdataset["Droits_Transmis_S"] = self.set_normed_data_default(self.get_match(r"DROITS\sTRANSMIS", n_groups=0), named_label)
    subdataset["Requisition_Publication_S"] = self.set_normed_data_default(self.get_match(r"R[E|É]QUISITION\s-\sPUBLICATION", n_groups=0), named_label)
    subdataset["Pouvoir_S"] = self.set_normed_data_default(self.get_match(r"POUVOIR", n_groups=0), named_label)
    subdataset["Certification_Attestation_S"] = self.set_normed_data_default(self.get_match(r"CERTIFICATION\sET\sATTESTATION", n_groups=0), named_label)
    subdataset["Autorisation_Destruction_Documents_S"] = self.set_normed_data_default(self.get_match(r"AUTORISATION\sDE\sDESTRUCTION\sDES\s\DOCUMENTS\sET\sPI[E|È]CES", n_groups=0), named_label)
    subdataset["Mention_Protection_Données_Personnelles_S"] = self.set_normed_data_default(self.get_match(r"MENTION\sSUR\sLA\sPROTECTION\sDES\sDONN[E|É]ES\sPERSONNELLES", n_groups=0), named_label)
    subdataset["Certification_Identite_S"] = self.set_normed_data_default(self.get_match(r"CERTIFICATION\sD’IDENTIT[E|É]", n_groups=0), named_label)
    return subdataset

  def get_json_notaire_1(self):
    '''Renvoie le JSON Dataset des informations du notaire associé à l'acte'''
    # L'AN DEUX MILLE VINGT-QUATRE,
    # LE date en lettres
    # A ville (département), adresse, au siège de l’Office Notarial, ci-après nommé,
    # Maître nom, Notaire Associé ou au sein de la Société Civile Professionnelle «dénomination »
    # titulaire d’un Office Notarial à ville (code postal), adresse

    self.set_flatten_text()

    subdataset = {}
    subdataset["Annee_1"] = self.set_normed_data_default(self.get_match(r"(L\s*\'\s*A\s*N.*?,)", n_groups=1), named_label="Annee_Lettre")
    subdataset["Date_1"] = self.set_normed_data_default(self.get_match(r",(.*?)\nA\s", n_groups=1), named_label="Date_Lettre")
    subdataset["Lieu_SON_1"] = self.set_normed_data_location(self.get_match(r"(?:\nA\s|le[\s\n]siège[\s\n]est[\s\n]à)(.*?)(?:\,*[\s\n]au[\s\n]siège|\,*\s*titulaire)", n_groups=0))
    subdataset["Lieu_ON_1"] = self.set_normed_data_location(self.get_match(r"(titulaire\n*\s*d\'*’*un\n*\s*[oO]ffice\n*\s*[nN]otarial.*?)A\sreçu\n*\s*le\n*\s*présent\n*\s*acte", n_groups=0))
    subdataset["Nom_Maitre_1"] = self.set_normed_data_name(self.get_match(r"(Ma[î|i]tre.*?)[nN]otaire", n_groups=1))
    subdataset["Type_Maitre_1"] = self.set_normed_data_default(self.get_match(r"[nN]otaire\s*\,*(.*?)de\n*\s*la\n*\s*[sS]ociété", n_groups=0), named_label="Type_Maitre")

    #print(subdataset["Annee_1"])
    n = self.update_flatten_text(first_quote="[nN]otaire")
    #subdataset["Nom_Societe_1"] = self.set_normed_data_nom_societe(self.get_match(r"(?:[\"«](.*?)[»\"]|[A-ZÉÈ\d-]{2,}\,)", index_before=n, n_groups=0))
    subdataset["Nom_Societe_1"] = self.set_normed_data_default(self.get_match(r"(?:[\"«](.*?)[»\"]|[A-ZÉÈ\d-]{3,}\,)", index_before=n, n_groups=0), named_label="Nom_Societe")

    return subdataset

  def get_json_qualif_juridique_2(self):
    '''Renvoie le JSON Dataset des qualifications juridiques l'acte'''

    # QUALIFICATION JURIDIQUE
    # A reçu le présent acte contenant ATTESTATION IMMOBILIERE APRES DECES à la requête de :…

    n = self.update_flatten_text(first_quote=r"pr[ée]sent\sacte", last_quote=r"la\srequ[êe]te")
    subdataset = {}
    subdataset["Type_Document_2"] = self.set_normed_data_default(self.get_match(r"([A-ZÉÈ]{3,}.*?)à", index_before=n, n_groups=1), named_label="Type_Document")

    self.set_flatten_text()

    if re.search(r"à[\s\n]la[\s\n]requ[êe]te[\s\n]de", self.flatten_text):
        #r"(Ci-après\snommés|en\svertu\sdes\spouvoirs)"
        n = self.update_flatten_text(first_quote=r"à[\s\n]la[\s\n]requ[êe]te[\s\n]de", last_quote="TEXTE\sAPPLICABLE", index_before=n)
        subdataset["Requerant_Acte_2"] = self.set_normed_data_qualif_juridique(self.get_matches(r"-(.*?)pr[ée]sent[\s\n]à[\s\n]l\’acte", index_before=n, n_groups=0))
    else:
        subdataset["Requerant_Acte_2"] = {
            "tokens": [],
            "tokens_labels": [],
            "positions_index": []
        }
    return subdataset

  def get_json_etat_civil_D_3(self):
    '''Renvoie le JSON Dataset de la comparutions clients (état civil) des parties prenantes de l'acte'''

    # PERSONNE DECEDEE
    # NOM en majuscules, prénoms en minuscules, domicile, date et lieu de naissance
    # Profession, nom du conjoint ou partenaire PACS, date et lieu du mariage, date divorce, veuvage
    # Nationalité, qualité de résident ou non au sens de la réglementation fiscale.

    # r"PERSONN(?:E|ES)\sD(?:E|É)C(?:E|É)D(?:E|É)(?:E|ES)"
    self.set_flatten_text()
    if re.search(r"PERSONNE[S]*\sD[EÉ]C[EÉ]D[EÉ]E[S]*", self.flatten_text):
       n = self.update_flatten_text(first_quote=r"PERSONNE[S]*\sD[EÉ]C[EÉ]D[EÉ]E[S]*", last_quote=r"D[E|É]VOLUTION\sSUCCESSORALE")

    #if r"PERSONNE[S]\sD[EÉ]C[EÉ]D[EÉ]E[S]" in self.flatten_text:
    #    print("TEST A")
    #    n = self.update_flatten_text(first_quote=r"PERSONNE[S]\sD[EÉ]C[EÉ]D[EÉ]E[S]", last_quote=r"D[E|É]VOLUTION\sSUCCESSORALE")
    else:
        print("PAS DE DECES")
        n = self.update_flatten_text(first_quote=r"D[E|É]VOLUTION\sSUCCESSORALE")
    subdataset = {}

    subdataset["Nom_D_3"] = self.set_normed_data_name(self.get_match(r"\b(Monsieur|Madame)\b(.*?)[A-ZÉÈÛÏ]{3,}\,\s", index_before=n, n_groups=0))
    subdataset["Domicile_D_3"] = self.set_normed_data_location(self.get_match(r"demeurant\n*\s*à(.*?\.)", index_before=n, n_groups=0))
    subdataset["Lieu_Naissance_D_33"] = self.set_normed_data_location(self.get_match(r"(\b(Né|Née)\b\sà.*?)(\.|le)", index_before=n, n_groups=0))
    subdataset["Date_Naissance_D_3"] = self.set_normed_data_date(self.get_match(r"\sle\s(.*?\.)", index_before=n, n_groups=0))
    # subdataset["Date_Naissance_D_3"] = self.set_normed_data_date(self.get_match(r"\sle\s(.*?)[\.\n]", index_before=n, n_groups=0))
    subdataset["Lieu_Deces_D_3"] = self.set_normed_data_location(self.get_match(r"(Décédé(?:e)?\s*à\s.*?)(\.|le)", index_before=n, n_groups=1))
    subdataset["Date_Deces_D_3"] = self.set_normed_data_date(self.get_match(r"Décédé[e]*\sà(?:.*)(\sle\s.*?\.)", index_before=n, n_groups=1))
    subdataset["Profession_D_3"] = self.set_normed_data_common_name(self.get_match(r"([Ee]n\n*\s*son\n*\s*vivant\s)(.*?\,)", index_before=n, n_groups=0), pattern=r"([Ee]n\n*\s*son\n*\s*vivant\s)(.*?\,)", named_label="Profession")
    subdataset["Situation_Maritale_D_3"] = self.set_normed_data_wedding(self.get_match(r"\b(?:Né|Née)\b\sà(?:.*?)\.(.*?)\.", index_before=n, n_groups=1)) #[4:] sur le groupe # TODO: A revoir
    #subdataset["Specification_Maritale_D_3"] = self.set_normed_data_default(self.get_match(r"\b(Né|Née)\b\sà([^\.]*)\.(.*?)\.(.*?)\.", index_before=n, n_groups=4), named_label="Spe_Marital")
    subdataset["Nationalite_D_3"] = self.set_normed_data_common_name(self.get_match(r"([Dd]e[\s\n][Nn]ationalit[ée])(.*?\.)", index_before=n, n_groups=0), pattern=r"([Dd]e[\s\n][Nn]ationalit[ée])(.*?\.)", named_label="Nationalite")
    subdataset["Qualite_Resident_D_3"] = self.set_normed_data_default(self.get_match(r"[Dd]e[\s\n][Nn]ationalit[ée].*?\.\s(.*?)[.(]", index_before=n, n_groups=1), named_label="Qual_Resident")

    ###subdataset["Ville_D_D_3"] = self.get_match(r"Décédé(?:e)? à\s(.*?)\(", index_before=n, n_groups=1)
    ###n = self.update_flatten_text(first_quote="Décédé")
    ###expression = self.get_matches(r"\((.*?)\)", index_before=n, n_groups=1)
    ###subdataset["Departement_D_D_3"] = expression[0][0][0], expression[1][0]
    ###subdataset["Pays_D_D_3"] = expression[0][1][0], expression[1][1]

    return subdataset

  def get_json_etat_civil_H_3(self):
    '''Renvoie le JSON Dataset de la comparutions clients (état civil) des parties prenantes de l'acte'''
    # Héritiers
    # NOM en majuscules, prénoms en minuscules, domicile, date et lieu de naissance
    # Profession, nom du conjoint ou partenaire PACS, date et lieu du mariage, date divorce, veuvage
    # Nationalité, qualité de résident ou non au sens de la réglementation fiscale.
    subdataset = {}
    n = self.update_flatten_text(first_quote=r"D[E|É]VOLUTION\sSUCCESSORALE", last_quote=r"\b(Prédécès\ssans\spostérité|QUALIT[E|É]S\sH[E|É]R[E|É]DITAIRES|D[E|É]SIGNATION)\b")

    subdataset["Nom_H_3"] = self.set_normed_data_name(self.get_matches(r"\b(Madame|Monsieur)\b(.*?)[A-ZÉÈ\\/-]{3,}\,", index_before=n, n_groups=0))
    subdataset["Profession_H_3"] = self.set_normed_data_common_name(self.get_matches(r"[A-ZÉÈ\\/-]{3,}\,([^\n]*?\,)", index_before=n, n_groups=0), pattern=r"(?:[A-ZÉÈ\\/-]{3,}\,)([^\n]*?\,)",named_label="Profession", set_prefixe=False)
    subdataset["Domicile_H_3"] = self.set_normed_data_location(self.get_matches(r"demeurant[\s\n]à(.*?)\.", index_before=n, n_groups=0))
    subdataset["Lieu_Naissance_H_3"] = self.set_normed_data_location(self.get_matches(r"\b(Né|Née)\b\sà\s(.*?)\sle\s", index_before=n, n_groups=0))
    subdataset["Date_Naissance_H_3"] = self.set_normed_data_date(self.get_matches(r"\b(?:Né|Née)\b\sà\s(?:.*?)(\sle\s.*?)[\.\n]", index_before=n, n_groups=1))
    subdataset["Situation_Maritale_H_3"] = self.set_normed_data_wedding(self.get_matches(r"\b(?:Né|Née)\b\sà(?:.*?)\.(.*?)\.", index_before=n, n_groups=1))
    #subdataset["Specification_Maritale_H_3"] = self.set_normed_data_default(self.get_matches(r"\b(Né|Née)\b\s(.*?)\.(.*?)\.(.*?)\.", index_before=n, n_groups=4), named_label="Spe_Marital")
    subdataset["Nationalite_H_3"] = self.set_normed_data_common_name(self.get_matches(r"([Dd]e[\s\n][Nn]ationalit[ée])(.*?\.)", index_before=n, n_groups=0), pattern=r"([Dd]e[\s\n][Nn]ationalit[ée])(.*?\.)", named_label="Nationalite")
    subdataset["Qualite_Resident_H_3"] = self.set_normed_data_default(self.get_matches(r"[Dd]e[\s\n][Nn]ationalit[ée].*?\.\s(.*?)[.(]", index_before=n, n_groups=1), named_label="Qual_Resident")
    subdataset["Relation_Familiale_H_3"] = self.set_normed_data_common_name(self.get_matches(r"[Ss](on|a)\s([^\s]*?\.)", index_before=n, n_groups=0), pattern=r"([Ss](?:on|a)\s)([^\s]*?\.)", named_label="Relation_Familiale")
    return subdataset

  def get_json_designation_4(self):
    '''Renvoie le JSON Dataset de la désignation complète des immeubles de l'acte'''

    # Nature du bien (terrain, parcelle, maison…), commune, adresse
    # Tableau cadastral avec préfixe (éventuellement), section, lieudit, surface
    #n = self.update_flatten_text(last_quote=r"[E|É]VALUATION")

    # n est le nombre de lignes au début de la partie filtrée
    # n = self.update_flatten_text(first_quote=r"\b(D[E|É]SIGNATION|IMMEUBLE[S]*\sDE\sCOMMUNAUTE|IMMEUBLE[S]*\sPROPRE[S]*)\b", last_quote=r"[E|É]VALUATION")
    n = self.update_flatten_text(first_quote=r"D[E|É]SIGNATION", last_quote=r"EFFET\sRELATIF")

    subdataset = {}
    pattern_prefixe_TC = r"\b([Ff]igurant\s(?:ainsi\s)*au\scadastre|L\'assiette\sde\sla\svolumétrie\sest\sla\ssuivante|cadastré)\b"

    # print("Flatten text : ", self.flatten_text[:1000])
    #subdataset["Total_Descriptif_P_4"] = self.get_match(r"\n*(.*?)" + pattern_prefixe_TC, index_before=n, n_groups=1)
    subdataset["Adresse_Propriete_4"] = self.set_normed_data_location(self.get_match(r"[A-ZÉÈ\\/-]{3,}(.*?)\,\n", index_before=n, n_groups=0))
    subdataset["Nature_P_4"] = self.set_normed_data_default(self.get_match(r"\,\n(.*?)" + pattern_prefixe_TC, index_before=n, n_groups=1), named_label="Nature_P")
    subdataset["Prefixe_TC_4"] = self.set_normed_data_default(self.get_match(pattern_prefixe_TC, index_before=n, n_groups=0), named_label="Prefixe_TC")

    #n = self.update_flatten_text(first_quote=pattern_prefixe_TC, last_n_lines=n)
    # Pattern pour extraire le contenu du tableau cadastral
    pattern = (
        r'([A-Z]+)'                 # Groupe 1: Premier mot en MAJUSCULES
        r'\s+(\d+)'                 # Groupe 2: Premier nombre
        r'\s([a-zA-Zéèîï\d\s]+[a-zA-Zéèîïûü]+)'  # Groupe 3: Entre le premier nombre et un mot en majuscules suivi d'un nombre
        r'\s(\d{2}\s[a-z]{2}\s\d{2}\s[a-z]{1}\s\d{2}\s[a-z]{2})' # Groupe 4: Motif spécifique commençant par un nombre
    )

    #subdataset["Tableau_Cadastral_4"] = self.get_matches(r"([A-Z]+)\s+(\d+)\s([a-zA-Zéèîï\d\s]+[a-zA-Zéèîïûü]+)\s(\d{2}\s[a-z]{2}\s\d{2}\s[a-z]{1}\s\d{2}\s[a-z]{2})", index_before=n, n_groups=0)
    subdataset["Tableau_Cadastral_4"] = self.set_normed_data_tableau_cadastral(self.get_matches(pattern, index_before=n, n_groups=0))
    return subdataset

  def get_json_evaluation_5(self):
    '''Renvoie le JSON Dataset de l'évaluation des immeubles associé à l'attestation de propriété'''
    #Evaluation en pleine propriété
    #Evaluation des quotités transmises (si bien transmis en partie et/ou démembrement)

    # TODO : Evaluation en Lettre

    n = self.update_flatten_text(first_quote=r"[E|É]VALUATION", last_quote=r"EFFET\sRELATIF")
    subdataset = {}
    #r"(\d+[\d\.\s]+\,\d{2}\sEUR)"
    #subdataset["Eval_Pleine_Propriete_5"] = self.get_match(r"\d+[\d\.\s]+\,\d{2}\s*(EUR|€)", index_before=n , n_groups=0)
    Evaluations_5 = self.get_matches(r"(\d+[\d\.\s]+\,\d{2}\s*(EUR|€))", index_before=n, n_groups=1)
    #Evaluations_5[:][-1], Evaluations_5[1][-1]
    subdataset["Eval_Pleine_Propriete_5"] = self.set_normed_data_argent([[Evaluations_5[0][0]], [Evaluations_5[1][0]]])
    subdataset["Eval_Quotites_Transmises_5"] = self.set_normed_data_argent([[Evaluations_5[0][-1]], [Evaluations_5[1][-1]]])
    return subdataset

  def get_json_etat_descriptif_6(self):
    '''Renvoie le JSON Dataset des références de publication de l'état descritif de division  de l'acte'''
    # Référence de publication de l'état descritif de division de ses évent mddificatifs
    # (Si bien soumis au régime de la copropriété)
    # r"(D[E|É]SIGNATION|IMMEUBLE[S]*\sDE\sCOMMUNAUTE|IMMEUBLE[S]*\sPROPRE[S]*)"
    n = self.update_flatten_text(first_quote=r"D[E|É]SIGNATION")
    subdataset = {}
    #r"\b(nue\n*-\n*propri[é|e]t[é|e]|copropri[é|e]t[é|e]|pleine\s*\n*-*\s*\n*propri[é|e]t[é|e])\b"
    subdataset["Regime_Propriete_6"] = self.set_normed_data_default(self.get_match(r"\b(nue\s*-\s*propri[ée]t[ée]|copropri[ée]t[ée]|pleine\s*-*\s*propri[ée]t[ée])\b", index_before=n, n_groups=1), "Regime_Propriete")
    n = self.update_flatten_text(last_quote=r"EFFET\sRELATIF", index_before=n)
    #print("Flatten text : ", self.flatten_text[:1000])
    # TODO : Corriger le pattern pour cas plusieurs effets relatifs
    subdataset["Reference_Etat_Descritif_Division_1_6"] = self.set_normed_data_reference(self.get_matches(r"volume\s*\d+\,*\s*num[ée]ro\s*\d+\.*", index_before=n, n_groups=0))
    return subdataset

  def get_json_effet_relatif_7(self):
    '''Renvoie le JSON Dataset des références de publication du titre immédiat de l'acte'''

    # Effet Relatif
    # Type d’acte, Notaire ayant reçu l’acte, lieu
    # SPF où l’acte a été publié ou mention « en cours de publication, date de publication,
    # volume, numéro.
    # Exemple : Acquisition suivant acte reçu par Maître XXX, notaire à XXX le XXX en cours de publication au service de la publicité foncière de CORBEIL 1.
    # / publié au service de la publicité foncière de XXX, le XXX, volume XXX, numéro XXX.

    self.set_flatten_text()
    if not re.search(r"EFFET\sRELATIF", self.flatten_text):
        return {}

    n = self.update_flatten_text(first_quote=r"EFFET\sRELATIF", last_quote="SERVITUDES")
    subdataset={}
    subdataset["Type_Acte_7"] = self.set_normed_data_default(self.get_matches(r"([^\n][^\n]*)\s*suivant\n*\s*acte\n*\s*reçu\n*\s*par", index_before=n, n_groups=1, replace="-"), named_label="Type_Acte")
    subdataset["Notaire_Effet_Relatif_7"] = self.set_normed_data_name(self.get_matches(r"suivant\n*\s*acte\n*\s*reçu\n*\s*par(.*?)\,", index_before=n, n_groups=1))
    subdataset["Lieu_Reception_7"] = self.set_normed_data_location(self.get_matches(r"lors\n*\s*[nN]otaire\n*\s*à(.*?)\d{4}\,", index_before=n, n_groups=0))
    subdataset["Lieu_Publicite_Fonciere_7"] = self.set_normed_data_location(self.get_matches(r"([Aa]u\s*\n*service\s*\n*de\n*\s*la\n*\s*publicité\n*\s*foncière\n*\s*de.*?)\,*\s*volume", index_before=n, n_groups=1))
    subdataset["Reference_Effet_Relatif_7"] = self.set_normed_data_reference(self.get_matches(r"volume\s*\d+\,*\s*num[ée]ro\s*\d+\.*", index_before=n, n_groups=0))
    subdataset["Date_Reception_7"] = self.set_normed_data_date(self.get_matches(r"[nN]otaire\n*\s*à.*?(\b(le|les)\b.*?\,)", index_before=n, n_groups=1))
    subdataset["Date_Publicite_Fonciere_7"] = self.set_normed_data_date(self.get_matches(r"publicité\n*\s*foncière\n*\s*de\n*\s*.*?(\b(le|les)\b.*?\,*)\svolume", index_before=n, n_groups=1))
    subdataset["Statut_Publication_7"] = self.set_normed_data_default(self.get_matches(r"\b(publié|en[\s\n]cours[\s\n]de[\s\n]publication)", index_before=n, n_groups=0), "Statut_Publication")
    return subdataset

  def get_json_droits_transmis_8(self):
    '''Renvoie le JSON Dataset des droits transmis de la succession'''

    # Le notaire soussigné atteste que, par suite du décès , les biens et droits immobiliers dont la désignation précède se sont trouvés transmis aux ayants droit en leur qualité
    # Madame XXX recueille « quotité »
    # Madame XXX recueille « quotité »

    n = self.update_flatten_text(first_quote=r"DROITS\sTRANSMIS", last_quote=r"R[E|É]QUISITION-PUBLICATION")
    subdataset={}
    subdataset["Nom_Droits_Transmis_8"] = self.set_normed_data_name(self.get_matches(r"([^\n][^\n]*)recueille", index_before=n, n_groups=0))
    subdataset["Valeur_Transmis_8"] = self.set_normed_data_valeur_transmis(self.get_matches(r"(recueille.*?)\b(Monsieur|Madame|REQUISITION\s-\sPUBLICATION)", index_before=n, n_groups=1))
    #subdataset["Droits_Transmis_Quotites_8"] = self.get_matches(r"recueille(.*?)\n", index_before=n, n_groups=1)
    return subdataset

  def get_json_requisition_publication_9(self):
    '''Renvoie le JSON Dataset de la Réquisition - Publication de l'acte'''
    # L’“ ayant droit ” requiert le notaire soussigné de dresser la présente attestation de propriété pour la faire publier.
    # La présente attestation de propriété sera publiée au service de la publicité foncière de XXX.
    # En fonction des dispositions à publier au fichier immobilier, la contribution de sécurité immobilière s'élève à la somme de XXX.
    # La taxe fixe sera perçue par ce service de la publicité foncière.

    n = self.update_flatten_text(first_quote=r"R[E|É]QUISITION\s*-\s*PUBLICATION", last_quote=r"POUVOIRS")
    subdataset = {}
    subdataset["Lieu_Requisition_PF_9"] = self.set_normed_data_location(self.get_match(r"[Aa]u[\n\s]service[\n\s]de[\n\s]la[\n\s]publicit[e|é][\n\s]fonci[e|è]re[\n\s]d(.*?)\.", index_before=n, n_groups=0))
    subdataset["Contribution_Securite_Immobiliere_9"] = self.set_normed_data_argent(self.get_match(r"contribution[\n\s]de[\n\s]s[e|é]curit[e|é][\n\s]immobili[e|è]re[\n\s]s\'[e|é]l[e|è]ve[\n\s]à[\n\s]la[\n\s]somme[\n\s]de(.*?)\.", index_before=n, n_groups=0))
    #subdataset["Taxe_Publicite_Fonciere_9"] = self.set_normed_data_argent(self.get_match(r"(TOTAL\s\d+[\d\.]+*\,\d{2})\n", index_before=n, n_groups=0))
    #subdataset["Contribution_Securite_Immobiliere_9"] = self.get_match(r"(\d+[\d\.]+\,\d{2})([^\d]*?)", index_before=n, n_groups=1)
    return subdataset

  def get_json_certification_attestation_10(self):

    # PAR SUITE DES FAITS ET ACTES SUS-ENONCES, le notaire soussigné certifie et atteste que les biens immobiliers faisant l’objet des présentes, appartiennent à :
    # Madame XXX à concurrence de XXX.
    # Madame XXX à concurrence de XXX.

    n = self.update_flatten_text(first_quote=r"CERTIFICATION\sET\sATTESTATION", last_quote=r"CERTIFICATION\sD’IDENTIT[E|É]")
    subdataset = {}
    subdataset["Personnes_Certifiees_10"] = self.set_normed_data_name(self.get_matches(r"\b(Madame|Monsieur)\b(.*?)[A-ZÉÈ-]{2,}", index_before=n, n_groups=0))
    subdataset["Limites_Concurrence_10"] = self.set_normed_data_valeur_transmis(self.get_matches(r"(à[\s\n]concurrence[\s\n]d.*?)(?:\b(Madame|Monsieur|AUTORISATION)\b)", index_before=n, n_groups=1))
    return subdataset

  def get_json_certification_identite_11(self):
    '''Renvoie le JSON Dataset de la Certification d'Identité devant apparaitre à la fin de l'acte'''

    # Le notaire soussigné certifie que l’identité complète des parties dénommées dans le présent document telle qu'elle est indiquée en tête des présentes à la suite de leur nom ou dénomination lui a été régulièrement justifiée.

    n = self.update_flatten_text(first_quote=r"CERTIFICATION\sD’IDENTIT[E|É]")
    subdataset = {}
    subdataset["Certification_Identite_11"] = self.set_normed_data_default(self.get_match(r"(.*?\.)", index_before=n, n_groups=0), named_label="Cert_Id_11")
    return subdataset

  def get_json_context(self):
    '''Renvoie le JSON Dataset d'autres informations'''
    subdataset = {}
    self.set_flatten_text()
    words = self.flatten_text.replace("\n", " ").split(" ")
    subdataset["Context"] = {
        "tokens": [],
        "tokens_labels": [],
        "positions_index": []
    }

    index_position = 0
    for word in words:
        if word not in self.token_visited:
            subdataset["Context"]["tokens"].append([word])
            subdataset["Context"]["tokens_labels"].append(["O"])
            subdataset["Context"]["positions_index"].append([index_position])
        index_position = index_position + len(word) + 1
    return subdataset

  # UTILS for Token Labeling for Classification

  def set_token_labels(self, token, named_label):
      '''Génère la liste des types de tokens selon la liste des tokens extraits'''
      token_labels = []
      token_labels.append("B-" + named_label)
      self.token_visited.append(token[0])
      num_words = len(token)
      if num_words >= 2:
          for j in range(num_words-2):
              token_labels.append("I-" + named_label)
              self.token_visited.append(token[j+1])
          token_labels.append("E-" + named_label)
          self.token_visited.append(token[-1])
      return token_labels

  def add_token(self, match, token, token_label, named_label, min=0, num_group=1):
      '''''Ajoute un token à la liste des tokens'''
      if match and len(match.group(num_group).strip()) > min:
          token = token + re.split(r'\s+', match.group(num_group).strip())
          token_label = token_label + self.set_token_labels(re.split(r'\s+', match.group(num_group).strip()), named_label)
      return token, token_label

  def get_start_index(self, found_match, tokens):
      '''Renvoie l'index de début de chaque token'''
      positions_index = []
      for i in range(len(found_match[0])):
          position_index = []
          split_found_match = re.split(r'\s+', found_match[0][i])
          for j in range(len(split_found_match)):
              try:
                  if split_found_match[j] in tokens[i]:
                      position_index.append(found_match[1][i][j])
              except:
                  #print("\nERROR") # TO DEBUG
                  #print(found_match[0])
                  #print(tokens)
                  pass
          positions_index.append(position_index)
      return positions_index

  def get_start_index_from_token(self, found_match, tokens):
      '''Renvoie l'index de début de chaque token'''
      positions_index = []
      for i in range(len(tokens)):
          position_index = []
          current_position = found_match[1][i][0]
          for j in range(len(tokens[i])):
              position_index.append(current_position)
              current_position = current_position + len(tokens[i][j])
          positions_index.append(position_index)
      return positions_index

  def set_normed_data_default(self, found_match, named_label):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match, tokens, tokens_labels = len(found_match[0]), [], []
    for i in range(num_match):
        token_list = re.split(r"\s+", found_match[0][i].strip())
        tokens.append(token_list)
        tokens_labels.append([token_label for token_label in self.set_token_labels(token_list, named_label)])

    positions_index = self.get_start_index(found_match, tokens)

    return {
        "tokens": tokens,
        "tokens_labels": tokens_labels,
        "positions_index": positions_index
    }

  def set_normed_data_location(self, found_match):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match, tokens, tokens_labels = len(found_match[0]), [], []
    for i in range(num_match):
        token, token_labels = [], []
        token, token_labels = self.add_token(re.search(r"(.*?)[A-ZÉÈ\s-]{3,}", found_match[0][i]), token, token_labels, "Prefixe_Lieu")
        token, token_labels = self.add_token(re.search(r"([A-ZÉÈ\s-]{3,})", found_match[0][i]), token, token_labels, "Ville")

        parenthese_match = re.findall(r"(\(.*?\)\,*(?:\s\d{5})*)", found_match[0][i])
        num_match = len(parenthese_match)

        if num_match >= 1:
            departement_token = str(parenthese_match[0])
            token.append(departement_token)
            token_labels = token_labels + self.set_token_labels([departement_token], "Departement")

        if num_match >= 2:
            pays_token = str(parenthese_match[1])
            token.append(pays_token)
            token_labels = token_labels + self.set_token_labels([pays_token], "Pays")

        #adresse_match = re.search(r"[A-ZÉÈ\s-]{3,}.*(?:(?<!\d),|\))\,*(.*?((?<!\d),|\.))", found_match[0][i])
        token, token_labels = self.add_token(re.search(r"[A-ZÉÈ\s-]{3,}.*(?:(?<!\d)\,|\))\,*(.*?((?<!\d)\,|\.))", found_match[0][i]), token, token_labels, "Adresse", min=1)
        token, token_labels = self.add_token(re.search(r"[\)\,]\,*\s(?:.*[\,\.])(.*?)$", found_match[0][i]), token, token_labels, "Suffixe_Lieu")

        tokens.append(token)
        tokens_labels.append(token_labels)

    positions_index = self.get_start_index(found_match, tokens)

    return {
        "tokens": tokens,
        "tokens_labels": tokens_labels,
        "positions_index": positions_index
    }

  def set_normed_data_date(self, found_match):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match, tokens, tokens_labels = len(found_match[0]), [], []
    for i in range(num_match):
        token, token_labels = [], []
        token, token_labels = self.add_token(re.search(r"(le\s)", found_match[0][i]), token, token_labels, "Prefixe_Date")
        token, token_labels = self.add_token(re.search(r"(\d{1,2}.*?)\s", found_match[0][i]), token, token_labels, "Jour")
        token, token_labels = self.add_token(re.search(r"le\s(?:.*)(\s[A-Za-zéèû]{3,})\s", found_match[0][i]), token, token_labels, "Mois")
        token, token_labels = self.add_token(re.search(r"(\d{4}\s*\.*)", found_match[0][i]), token, token_labels, "Annee")
        tokens.append(token)
        tokens_labels.append(token_labels)

    positions_index = self.get_start_index(found_match, tokens)

    return {
        "tokens": tokens,
        "tokens_labels": tokens_labels,
        "positions_index": positions_index
    }

  def set_normed_data_name(self, found_match):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match, tokens, tokens_labels = len(found_match[0]), [], []
    for i in range(num_match):
        match = re.search(r"\b(Monsieur|Madame|Ma[î|i]tre)\b(.*?\,*)([A-ZÉÈ\\/-]{3,}\,*)", found_match[0][i])
        if match:
            token_group1, token_group2, token_group3 = re.split(r"\s+", match.group(1).strip()), re.split(r"\s+", match.group(2).strip()), re.split(r"\s+", match.group(3).strip())
            civilite, prenom, nom = self.set_token_labels(token_group1, "Civilite"), self.set_token_labels(token_group2, "Prenom"), self.set_token_labels(token_group3, "Nom")
            tokens.append(token_group1 + token_group2 + token_group3),
            tokens_labels.append(civilite + prenom + nom)

    positions_index = self.get_start_index(found_match, tokens)

    return {
        "tokens": tokens,
        "tokens_labels": tokens_labels,
        "positions_index": positions_index
    }

  def set_normed_data_nom_societe(self, found_match):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    if len(found_match[0]) == 0:
        return {"tokens": [], "tokens_labels": [], "positions_index": []}
    else:
        notaires = re.findall(r"([A-Za-z-éèîïÉÈ]+\,*)\s([A-ZÉÈ]{2,}\,*)\s", found_match[0][0])
        majuscules = re.findall(r"([A-ZÉÈ\d-]{2,}\,*)", found_match[0][0])
        prenom_labels = [notaire[0] for notaire in notaires]
        nom_labels = [notaire[1] for notaire in notaires]
        tokens = found_match[0][0].split(" ")
        tokens_labels = []
        for word in tokens:
            if word in prenom_labels:
                tokens_labels.append("Prenom_Societe")
            elif word in nom_labels:
                tokens_labels.append("Nom_Societe")
            elif word in ["\"", "«", "»"]:
                tokens_labels.append("Encadrement_Societe")
            elif word in majuscules:
                tokens_labels.append("Nom_Societe_Maj")
            else:
                tokens_labels.append("Nom_Societe_Min")
            self.token_visited.append(word)

        positions_index = self.get_start_index(found_match, tokens)

        return {
            "tokens": tokens,
            "tokens_labels": tokens_labels,
            "positions_index": positions_index
        }

  def set_normed_data_qualif_juridique(self, found_match):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match = len(found_match[0])
    if num_match == 0:
        return {"tokens": [], "tokens_labels": [], "positions_index": []}
    else:
        tokens = []
        tokens_labels = []
        for i in range(num_match):
            requerant = re.search(r"-\s*([Mm]adame|[Mm]onsieur)(.*?)([A-ZÉÈ]{2,})\,*", found_match[0][i])
            token_group1, token_group2, token_group3 = re.split(r"\s+", requerant.group(1).strip()), re.split(r"\s+", requerant.group(2).strip()), re.split(r"\s+", requerant.group(3).strip())
            civilite_requerant, prenom_requerant, nom_requerant = self.set_token_labels(token_group1, "Civilite"), self.set_token_labels(token_group2, "Prenom"), self.set_token_labels(token_group3, "Nom")
            tokens.append(token_group1 + token_group2 + token_group3)
            tokens_labels.append(civilite_requerant  + prenom_requerant + nom_requerant)

        positions_index = self.get_start_index(found_match, tokens)

        return {
            "tokens": tokens,
            "tokens_labels": tokens_labels,
            "positions_index": positions_index
        }

  def set_normed_data_nationalite(self, found_match):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match, tokens, tokens_labels = len(found_match[0]), [], []
    for i in range(num_match):
        token, token_labels = [], []
        token, token_labels = self.add_token(re.search(r"([Dd]e[\s\n][Nn]ationalit[ée])", found_match[0][i]), token, token_labels, "Prefixe_Nationalite")
        token, token_labels = self.add_token(re.search(r"[Dd]e[\s\n][Nn]ationalit[ée](.*?)$", found_match[0][i]), token, token_labels, "Nationalite")

        tokens.append(token)
        tokens_labels.append(token_labels)

    positions_index = self.get_start_index(found_match, tokens)

    return {
        "tokens": tokens,
        "tokens_labels": tokens_labels,
        "positions_index": positions_index
    }

  def set_normed_data_wedding(self, found_match):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match, tokens, tokens_labels = len(found_match[0]), [], []
    for i in range(num_match):
        token, token_labels = [], []
        # situation_match = re.search(r"(.*?)\b(Monsieur|Madame)\b", found_match[0][i])
        # token = token + situation_match.group(1).strip().split(" ") if situation_match else token
        # token_labels = token_labels + self.set_token_labels(situation_match.group(1).strip().split(" "), "Situation_Marital") if situation_match else token_labels

        token, token_labels = self.add_token(re.search(r"(.*?)\b(Monsieur|Madame)\b", found_match[0][i]), token, token_labels, "Situation_Marital")
        token, token_labels = self.add_token(re.search(r"(\b(Monsieur|Madame)\b)", found_match[0][i]), token, token_labels, "Civilite")
        token, token_labels = self.add_token(re.search(r"\b(?:Monsieur|Madame)\b(.*?)[A-ZÉÈ]{2,}", found_match[0][i]), token, token_labels, "Prenom_Conjoint")
        token, token_labels = self.add_token(re.search(r"([A-ZÉÈ]{2,})", found_match[0][i]), token, token_labels, "Nom_Conjoint")
        token, token_labels = self.add_token(re.search(r"[A-ZÉÈ]{2,}\s(.*?)", found_match[0][i]), token, token_labels, "Suffixe_SM", min=1)

        tokens.append(token)
        tokens_labels.append(token_labels)

    positions_index = self.get_start_index(found_match, tokens)

    return {
        "tokens": tokens,
        "tokens_labels": tokens_labels,
        "positions_index": positions_index
    }

  def set_normed_data_common_name(self, found_match, pattern = "", named_label = "", set_prefixe = True):
    '''Spécifie les types de tokens_labels associé au found_match'''
    #Ex : {"tokens": ["Ceci", "est", "une", "phrase"], "tokens_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}

    num_match, tokens, tokens_labels = len(found_match[0]), [], []
    for i in range(num_match):
        token, token_labels = [], []
        match = re.search(r"{}".format(pattern), found_match[0][i])
        if match :
            if set_prefixe:
                prefixe = re.split(r"\s+", match.group(1).strip())
                token = token + prefixe
                token_labels = token_labels + self.set_token_labels(prefixe, f"Prefixe_{named_label}")

                common_name = re.split(r"\s+", match.group(2).strip())
                token = token + common_name
                token_labels = token_labels + self.set_token_labels(common_name, named_label)
            else:
                common_name = re.split(r"\s+", match.group(1).strip())
                token = token + common_name
                token_labels = token_labels + self.set_token_labels(common_name, named_label)

        tokens.append(token)
        tokens_labels.append(token_labels)

    positions_index = self.get_start_index(found_match, tokens)

    return {
        "tokens": tokens,
        "tokens_labels": tokens_labels,
        "positions_index": positions_index
    }

  def set_normed_data_argent(self, found_match):
      '''Met en bonne forme les données extraites'''
      #Ex : {"token": ["Ceci", "est", "une", "phrase"], "token_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}
      num_match, tokens, tokens_labels = len(found_match[0]), [], []

      for i in range(num_match):
        token, token_labels = [], []

        token, token_labels = self.add_token(re.search(r"([^\d]+?)\d+[\.\s]\d+\,\d{2,}[\s\n\t]", found_match[0][i]), token, token_labels, "Prefixe_Argent")
        token, token_labels = self.add_token(re.search(r"(\d+[\.\s])\d+\,\d{2,}[\s\n\t]", found_match[0][i]), token, token_labels, "Millier_Euro")
        token, token_labels = self.add_token(re.search(r"[\.\s\(](\d+?\,)\d{2,}[\s\n\t]", found_match[0][i]), token, token_labels, "Euro")
        token, token_labels = self.add_token(re.search(r"[\.\s\(]\d+\,(\d{2,})[\s\n\t]", found_match[0][i]), token, token_labels, "Centime_Euro")
        token, token_labels = self.add_token(re.search(r"\b(EUR|€|eur\)|eur)\b", found_match[0][i]), token, token_labels, "Devise")
        token, token_labels = self.add_token(re.search(r"\,\d{2,}\s*[EUReur€]+(.*?)$", found_match[0][i]), token, token_labels, "Suffixe_Argent")

        tokens.append(token)
        tokens_labels.append(token_labels)

      positions_index = self.get_start_index_from_token(found_match, tokens)

      return {
          "tokens": tokens,
          "tokens_labels": tokens_labels,
          "positions_index": positions_index
      }

  def set_normed_data_reference(self, found_match):
      '''Met en bonne forme les données extraites'''
      #Ex : {"token": ["Ceci", "est", "une", "phrase"], "token_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}
      num_match, tokens, tokens_labels = len(found_match[0]), [], []
      for i in range(num_match):
        token, token_labels = [], []
        token, token_labels = self.add_token(re.search(r"(volume.*?)num", found_match[0][i]), token, token_labels, "Volume_Ref")
        token, token_labels = self.add_token(re.search(r"(num.*?)$", found_match[0][i]), token, token_labels, "Numero_Ref")

        tokens.append(token)
        tokens_labels.append(token_labels)

      positions_index = self.get_start_index(found_match, tokens)

      return {
          "tokens": tokens,
          "tokens_labels": tokens_labels,
          "positions_index": positions_index
      }

  def set_normed_data_valeur_transmis(self, found_match):
      '''Met en bonne forme les données extraites'''
      #Ex : {"token": ["Ceci", "est", "une", "phrase"], "token_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}
      num_match, tokens, tokens_labels = len(found_match[0]), [], []
      for i in range(num_match):
        token, token_labels = [], []

        token, token_labels = self.add_token(re.search(r"\b(recueille|[àa][\s\n]concurrence)\b", found_match[0][i]), token, token_labels, "Prefixe_Part")
        token, token_labels = self.add_token(re.search(r"\b(?:recueille|[àa][\s\n]concurrence)\b(.*?)\b(?:\(|\sen\s)\b", found_match[0][i]), token, token_labels, "Part_En_Lettre")
        token, token_labels = self.add_token(re.search(r"(\(.*?\))", found_match[0][i]), token, token_labels, "Part`_En_Chiffre")
        token, token_labels = self.add_token(re.search(r"\b(?:\(|\sen\s)\b(.*?)$", found_match[0][i]), token, token_labels, "Type_Propriete")

        tokens.append(token)
        tokens_labels.append(token_labels)

      positions_index = self.get_start_index(found_match, tokens)

      return {
          "tokens": tokens,
          "tokens_labels": tokens_labels,
          "positions_index": positions_index
      }

  def set_normed_data_tableau_cadastral(self, found_match):
      '''Met en bonne forme les données extraites'''
      #Ex : {"token": ["Ceci", "est", "une", "phrase"], "token_labels": ["Pronom", "Verbe", "Déterminant", "Nom_Commun"], "positions_index": 100}
      num_match, tokens, tokens_labels = len(found_match[0]), [], []
      for i in range(num_match):
        token, token_labels = [], []

        token, token_labels = self.add_token(re.search(r"([A-Z]+)", found_match[0][i]), token, token_labels, "Section")
        token, token_labels = self.add_token(re.search(r"\s+(\d+)", found_match[0][i]), token, token_labels, "Numero")
        token, token_labels = self.add_token(re.search(r"[A-Z]+\s+\d+\s([a-zA-Zéèîï\d\s]+[a-zA-Zéèîïûü]+)\s\d{2}\s[a-z]{2}\s\d{2}\s[a-z]{1}\s\d{2}\s[a-z]{2}", found_match[0][i]), token, token_labels, "Lieudit")
        token, token_labels = self.add_token(re.search(r"\s(\d{2}\s[a-z]{2}\s\d{2}\s[a-z]{1}\s\d{2}\s[a-z]{2})", found_match[0][i]), token, token_labels, "Surface")

        tokens.append(token)
        tokens_labels.append(token_labels)

      positions_index = self.get_start_index(found_match, tokens)

      return {
          "tokens": tokens,
          "tokens_labels": tokens_labels,
          "positions_index": positions_index
      }

  # Display JSON Datasets

  def get_json_dataset(self):
    return self.json_dataset

  def display_json_dataset(self):
    #print("\nJSON DATASET : {}".format(self.get_json_dataset()))
    print("\nJSON DATASET : ")
    pprint(self.get_json_dataset())
    return

  def get_json_subdataset(self, num_json):
    '''Renvoie le json associé à son numéro num_json'''
    if num_json == 0:
      return self.get_json_sections()
    if num_json == 1:
      return self.get_json_notaire_1()
    if num_json == 2:
      return self.get_json_qualif_juridique_2()
    if num_json == 3:
      return self.get_json_etat_civil_3()
    if num_json == 4:
      return self.get_json_designation_4()
    if num_json ==5:
      return self.get_json_evaluation_5()
    if num_json==6:
      return self.get_json_etat_descriptif_6()
    if num_json==7:
      return self.get_json_effet_relatif_7()
    if num_json==8:
      return self.get_json_droits_transmis_8()
    if num_json==9:
      return self.get_json_requisition_publication_9()
    if num_json==10:
      return self.get_json_certification_attestation_10()
    if num_json==11:
      return self.get_json_certification_identite_11()
    if num_json==12:
      return self.get_json_context()
    return {}

  def display_json_subdataset(self, nums_sections):
    '''Affiche les JSON datasets associés aux numéros de sections en entrée'''
    if 0 in nums_sections:
      print("\n0) JSON Dataset - Sections :\n")
      pprint(self.get_json_sections())
    if 1 in nums_sections:
      print("\n1) JSON Dataset - Notaire :\n")
      pprint(self.get_json_notaire_1())
    if 2 in nums_sections:
      print("\n2) JSON Dataset - Qualification Juridique :\n")
      pprint(self.get_json_qualif_juridique_2())
    if 3 in nums_sections:
      print("\n3) JSON Dataset - Etat Civil :\n")
      pprint(self.get_json_etat_civil_D_3())
      print("\n")
      pprint(self.get_json_etat_civil_H_3())
    if 4 in nums_sections:
      print("\n4) JSON Dataset - Designation :\n")
      pprint(self.get_json_designation_4())
    if 5 in nums_sections:
      print("\n5) JSON Dataset - Evaluation :\n")
      pprint(self.get_json_evaluation_5())
    if 6 in nums_sections:
      print("\n6) JSON Dataset - Références de publication de l'état descriptif de division\n")
      pprint(self.get_json_etat_descriptif_6())
    if 7 in nums_sections:
      print("\n7) JSON Dataset - Références de publication du titre immédiat :\n")
      pprint(self.get_json_effet_relatif_7())
    if 8 in nums_sections:
      print("\n8) JSON Dataset - Droits Transmis :\n")
      pprint(self.get_json_droits_transmis_8())
    if 9 in nums_sections:
      print("\n9) JSON Dataset - Réquisition - Publication :\n")
      pprint(self.get_json_requisition_publication_9())
    if 10 in nums_sections:
      print("\n10) JSON Dataset - Certification et Attestation :\n")
      pprint(self.get_json_certification_attestation_10())
    if 11 in nums_sections:
      print("\n11) JSON Dataset - Certification d'Identité :\n")
      pprint(self.get_json_certification_identite_11())
    if 12 in nums_sections:
      print("\n11) JSON Dataset - Contexte :\n")
      pprint(self.get_json_context())
    return

  def display_first_info(self):
    '''Affiche les premières informations extraites'''
    print("\nFirst Info :\n")
    print(self.texts[0])
    return

def get_json_dataset_from_pdf(path_data_folder, path_json_folder, clean_dataset=False, selected_datasets=[], display_sections=[], num_max_data=0, print_first_info=False):
    '''Extrait tous les pdf présents dans le dossier (+sous-dossier) associé au chemin path_folder'''

    paths_pdf = pathlib.Path(path_data_folder).rglob('*.pdf')
    paths_pdf = [str(path_pdf) for path_pdf in paths_pdf]
    path_json = path_json_folder + "/attestation_propriete_train_dataset.json"

    json_dataset = {}

    try:
        if (clean_dataset):
            with open(path_json, 'w') as json_file:
                json.dump({}, json_file)
            print("Le fichier JSON situé à l'emplacement {} a été nettoyé.".format(path_data_folder))
        else:
            print("Import Trained Dataset...")
            with open(path_json, 'r') as json_file:
                json_dataset = json.load(json_file)
    except Exception as e:
        print("Une erreur est survenue:", str(e))
    else:
        print("Création JSON file au :",path_data_folder)
        json_dataset["Type PDF"] = "Attestation de propriété"
        json_dataset["Data"] = {}

    print("Extraction des données... {} PDF trouvés.".format(len(paths_pdf)))

    for i, path_pdf in tqdm(enumerate(paths_pdf)):
        if (i < num_max_data or num_max_data == 0) :
            if (i in selected_datasets or len(selected_datasets) == 0):
                print("\nAnalysis of {}".format(os.path.basename(path_pdf)))
                sequence_classification = Classifier_Attestation_Propriete(path_pdf)
                #if sequence_classification.is_text():
                if print_first_info:
                    sequence_classification.display_first_info()
                sequence_classification.import_pdf()
                json_data = sequence_classification.extract()
                json_dataset["Data"][os.path.basename(path_pdf)] = json_data
                sequence_classification.display_json_subdataset(display_sections)
        else:
            print("\nBreaking at iteration {}...".format(i))
            break

    with open(path_json, 'w') as json_file:
        json.dump(json_dataset, json_file, indent=4)
    return json_dataset

if __name__ == "__main__":

    # Extraction des données PDF localisées dans le dossier data

    path_data_folder = '/content/drive/My Drive/IA/IA Notaire - BellVision/Data'
    path_repo_train_dataset = '/content/drive/My Drive/IA/IA Notaire - BellVision/Data'
    #path_data_folder = os.path.join(os.path.dirname(__file__), '..', 'data')
    #path_repo_train_dataset = os.path.join(os.path.dirname(__file__), '..', 'data')

    #selected_datasets = range(100, 203) # API TANGAMA = 28
    #selected_datasets = range(70, 182)
    selected_datasets = range(182)

    display_sections = [] # Jusqu'à 12 ici
    num_max_data = 0
    print_first_info = False

    json_dataset = get_json_dataset_from_pdf(path_data_folder, path_repo_train_dataset, clean_dataset=True, selected_datasets=selected_datasets, display_sections=display_sections, num_max_data=num_max_data, print_first_info=print_first_info)
    # pprint(json_dataset)

    # TODO : Reception_Acte_7
    # TODO : 96 Acte avec plusieurs désignations
    # TODO : Annee_1 pour 1.pdf

    # TODO : A CHECKER
    # TODO : Lieu_Reception_7
    # TODO : Lieu Siege Office Notarial

    # Test Unitaire dataset
    #path_pdf = os.path.join(os.path.dirname(__file__), "API TANGAMA.pdf")
    #json_dataset = Sequence_Classifier_Attestation_Propriete(path_pdf)
    #json_dataset.extract()
    #json_dataset.display_json_dataset()
    #json_dataset.display_json_subdataset(display_sections)

    # TODO : Remplacer "lines" par "positions_index" de l'expression dans une liste pour matches
    # Concaténer les tokens_labels dans Training Model pour créer du contexte lors de l'entrainement
    # On entraine le modèle, page par page
    # Détecter les expressions a considérer comme inutiles : Tout - Moins ce qu'on a détecter

    # TODO : Remplacer les match.group(1).strip().split(" ") par une variable local pour éviter les repetitions dans les sets



Le fichier JSON situé à l'emplacement /content/drive/My Drive/IA/IA Notaire - BellVision/Data a été nettoyé.
Création JSON file au : /content/drive/My Drive/IA/IA Notaire - BellVision/Data
Extraction des données... 232 PDF trouvés.


0it [00:00, ?it/s]


Analysis of API ATHON.pdf


1it [00:03,  3.09s/it]


Analysis of API BIDI.pdf


...

180it [11:14,  3.83s/it]


Analysis of 2.pdf


181it [11:16,  3.37s/it]


Analysis of 3.pdf


232it [11:19,  2.93s/it]


### Training Model

In [7]:
# JSON Dataset builder - Main
import datetime
import os
# from dotenv import load_dotenv
import pathlib
import json
import pdfplumber
import re
import numpy as np
import matplotlib.pyplot as plt
from evaluate import load #, evaluator # TO STUDY
from sklearn.metrics import precision_score, recall_score, f1_score
#import torch
#print(torch.__version__)

from pprint import pprint
from functools import reduce
#import pandas as pd
from tqdm import tqdm
from datasets import load_metric, Dataset

# IMPORTATION DES MODELES PRE ENTRAINE
from transformers import Trainer, TrainingArguments, TrainerCallback, pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM, AutoConfig
from transformers import DataCollatorWithPadding, DataCollatorForTokenClassification
#from transformers import DistilBertForSequenceClassification, DistilBertForTokenClassification, DistilBertTokenizerFast

#from scikit-learn.preprocessing import LabelEncoder

# Pour m'authentifier à une API
#from transformers import HfFolder, HfApi
#HfFolder.save_token("hf_api_token")

# Charger le fichier .env
#load_dotenv()

# API KEY
# api_key = os.environ.get("HUGGINGFACE_API_KEY")

class Classification_Model:
    # https://huggingface.co/mistralai/Mistral-7B-v0.1

    # Possible models :
    # - distilbert-base-cased
    # - roberta-large
    # - bert-base-cased
    # - dslim/bert-base-NER # Token Classification Based
    # - HuggingFaceFW/fineweb-edu-classifier
    # - ProsusAI/finbert # Sequence Classification Based

    # - TinyLlama/llama-7b-hf
    # - TinyLlama/TinyLlama-1.1B-Chat-v1.0
    # - meta-llama/Meta-Llama-3-8B-Instruct
    # - electra-base-cased
    # - openai/whisper-large-v3
    # - mistralai/Mistral-7B-Instruct-v0.3
    # - mistralai/Codestral-22B-v0.1 # Specific pour le code
    # - mistralai/Mistral-7B-v0.3 -- Require Python 3.10.11
    # - mistralai/Mistral-7B-v0.1

    def __init__(self, model_name="distilbert-base-cased"):
        self.model_name = model_name
        # self.path_dataset =  os.path.join(os.path.dirname(__file__), '..', 'data')
        # self.path_loss = os.path.join(os.path.dirname(__file__), '..', 'model', 'logs.json')
        self.path_dataset = '/content/drive/My Drive/IA/IA Notaire - BellVision/Data'
        self.path_loss = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model/logs.json'
        self.path_model = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model'
        self.normed_dataset = None
        self.snips_dataset = {"train": [], "test": []}
        self.token_classification_dataset = {"train": [], "test": []}
        self.sequence_classification_dataset = {"train": [], "test": []}
        self.tokenizer = None
        self.dataset_labels = None
        self.token_classification_model = None
        self.sequence_classification_model = None
        self.token_data_collator = None
        self.sequence_data_collator = None
        self.logs = {}


    # SET DATASET FOR TRAINING

    def load_dataset(self, json_name="attestation_propriete_train_dataset.json"):
        '''Function to load the dataset'''
        path_json = self.path_dataset + "/" + json_name
        with open(path_json) as json_file:
            self.normed_dataset = json.load(json_file)

    def get_token_classification_tokenized_dataset(self, n_range=0):
        '''Function to get the token classification dataset for training'''
        return self.token_classification_dataset["train"][n_range]

    def get_sequence_classification_tokenized_dataset(self, n_range=0):
        '''Function to get the sequence classification dataset for training'''
        return self.sequence_classification_dataset["train"][n_range]

    def set_train_dataset(self):

        #On veut travailler avec l'équivalent tokenizé pour chaque sequence selon le modèle de BERT pré-entrainé
        #Celui ci est se trouve dans 'input_ids'
        #Une matrice d'attention 'attention_mask' est calculé de même pour comprendre le contexte
        #'labels' représente l'équivalent de 'token-label' prenant en compte le modèle BERT
        #En effet, certains mots se décompose en plusieurs tokens suivant le dictionnaire de BERT

        print("\nSet Dataset for Sequence and Token Classification")

        # Input Dataset en forme json
        utterances, tokenized_utterances, labels_for_tokens, positions_indexes, sequence_labels = [], [], [], [], []

        for keyPDF, valuePDF in self.normed_dataset["Data"].items():
            utterances_per_pdf, tokenized_utterances_per_pdf, labels_for_tokens_per_pdf, positions_index_per_pdf, sequence_labels_per_pdf = [], [], [], [], []
            for keySection, valueSection in valuePDF.items():
                for keyAttribut, valueAttribut in valueSection.items():
                    for keyData, valueData in valueAttribut.items():
                        # print("valueData : ", valueData)
                        if type(valueData) == int:
                            valueData = [valueData]
                            print("valueData : ", valueData)
                        for value in valueData:
                            if keyData == 'tokens':
                                utterances_per_pdf.append(' '.join(value))
                                tokenized_utterances_per_pdf.append(value)
                                sequence_labels_per_pdf.append(keyAttribut)
                            if keyData == 'tokens_labels':
                                labels_for_tokens_per_pdf.append(value)
                            if keyData == 'positions_index':
                                positions_index_per_pdf.append(value)
            utterances.append(utterances_per_pdf)
            tokenized_utterances.append(tokenized_utterances_per_pdf)
            labels_for_tokens.append(labels_for_tokens_per_pdf)
            positions_indexes.append(positions_index_per_pdf)
            sequence_labels.append(sequence_labels_per_pdf)

        pdf_example = 0
        sequence_example = 150
        print("\nExample of the dataset :")
        print("PDF n°{} / Sequence example : {}".format(pdf_example, sequence_example))
        print("Sequence label : ", sequence_labels[pdf_example][sequence_example], "/ Total Length Per Pdf : ", len(sequence_labels[pdf_example]))
        print("Utterance : ", utterances[pdf_example][sequence_example], "/ Total Length Per Pdf : ", len(utterances[pdf_example]))
        print("Tokens : ", tokenized_utterances[pdf_example][sequence_example], "/ Total Length Per Pdf : ", len(tokenized_utterances[pdf_example]))
        print("Token labels : ", labels_for_tokens[pdf_example][sequence_example], "/ Total Length Per Pdf : ", len(labels_for_tokens[pdf_example]))
        print("Positions Indexes : ", positions_indexes[pdf_example][sequence_example], "/ Total Length Per Pdf : ", len(positions_indexes[pdf_example]))


        # On aplatit sequence_labels pour en extraire la liste unique des éléments qui le compose
        flatten_sequence_labels = [label for sequence in sequence_labels for label in sequence]
        self.unique_sequence_labels = list(set(flatten_sequence_labels))

        #Convertit les type de sequence de str à int en considérant leur indice
        #Crée un nombre entier unique (token) pour chaque type de sequence
        int_sequence_labels = [self.unique_sequence_labels.index(label) for label in flatten_sequence_labels]
        print(f'\nThere are {len(self.unique_sequence_labels)} unique sequence labels :\n{self.unique_sequence_labels}')

        # On aplatit labels_for_tokens pour en extraire la liste unique des éléments qui le compose
        flatten_labels_for_tokens = [label for sequences in labels_for_tokens for sequence in sequences for label in sequence]
        #self.unique_token_labels = list(set(reduce(lambda x, y: x + y, flatten_labels_for_tokens)))
        self.unique_token_labels = list(set(flatten_labels_for_tokens))

        #Crée un nombre entier unique (token) pour chaque type de token
        int_labels_for_tokens = [[[self.unique_token_labels.index(a) for a in b] for b in labels_for_tokens[i]] for i in range(len(labels_for_tokens))]
        flatten_int_labels_for_token = [item for sublist in int_labels_for_tokens for item in sublist]
        print(f'\nThere are {len(self.unique_token_labels)} unique token labels :\n{self.unique_token_labels}')

        tokens, token_labels, positions_indexes = self.sort_dataset(tokenized_utterances, int_labels_for_tokens, positions_indexes)

        # TODO : Prendre en compte le numero de ligne pour chaque token
        self.token_classification_dataset = Dataset.from_dict(
            dict(
                # utterance=utterances,
                tokens=tokens,
                token_labels=token_labels,
            )
        )

        print("\nToken Classification Dataset before alignment : PDF n°{}\n{}\n".format(
            pdf_example, self.token_classification_dataset[pdf_example]['token_labels'])
        )

        # Map our snip dataset to be for token classification
        self.token_classification_dataset = self.token_classification_dataset.map(
            self.tokenize_and_align_labels,
            batched=True
            )

        #On entraine le dataset en considérant 80% des données chargés
        #Les derniers 20% seront utilisés pour tester les performances de notre modèle
        self.token_classification_dataset  = self.token_classification_dataset.train_test_split(test_size=0.2)

        print("\nToken Classification Dataset after alignment : PDF n°{}\n{}\n".format(
            pdf_example, self.token_classification_dataset['train'][pdf_example]['token_labels'])
        )




        #We delete useless informations to improve algorithm efficency for token classification
        self.token_classification_dataset['train'] = self.token_classification_dataset['train'].remove_columns(
            ['tokens', 'token_labels']
        )
        self.token_classification_dataset['test'] = self.token_classification_dataset['test'].remove_columns(
            ['tokens', 'token_labels']
        )
        print("\nToken Classification Tokenized Snips Cleaned Formatted :\n{}\n".format(
            self.token_classification_dataset)
        )

        # Same for Sequence Classification

        flatten_utterances = [item for sublist in utterances for item in sublist]
        flatten_tokens = [item for sublist in tokenized_utterances for item in sublist]

        # Map our snip dataset to be for sequence classification
        self.sequence_classification_dataset = Dataset.from_dict(
            dict(
                utterance=flatten_utterances,
                label=int_sequence_labels,
                tokens=flatten_tokens,
                token_labels=flatten_int_labels_for_token,
            )
        )

        print("\nSequence Classification Dataset before alignment : Sequence n°{}\n{}\n".format(
            sequence_example, self.sequence_classification_dataset[sequence_example])
        )

        self.sequence_classification_dataset = self.sequence_classification_dataset.map(
            self.preprocess_function,
            batched=True
        )

        #On entraine le dataset en considérant 80% des données chargés
        #Les derniers 20% seront utilisés pour tester les performances de notre modèle
        self.sequence_classification_dataset  = self.sequence_classification_dataset.train_test_split(test_size=0.2)

        print("\nSequence Classification Dataset after alignment : Sequence n°{}\n{}\n".format(
            sequence_example, self.sequence_classification_dataset['train'][sequence_example])
        )

        #We delete useless informations to improve algorithm efficency for sequence classification
        self.sequence_classification_dataset['train'] = self.sequence_classification_dataset['train'].remove_columns(
            ['utterance', 'tokens', 'token_labels'] #, 'line_utterances'
        )
        self.sequence_classification_dataset['test'] = self.sequence_classification_dataset['test'].remove_columns(
            ['utterance', 'tokens', 'token_labels'] # , 'line_utterances'
        )
        print("\nSequence Classification Tokenized Snips Cleaned Formatted :\n{}".format(self.sequence_classification_dataset))
        print("\nDataset In Place !")
        return

    def sort_dataset(self, tokenized_utterances, labels_for_tokens, positions_indexes):
        '''Function to sort the dataset'''

        num_pdf = len(tokenized_utterances)
        tokens, token_labels, positions = [], [], []

        print("\nBefore sorting :")
        print(f"Tokens : {tokenized_utterances[0][:50]}")
        print(f"Token Labels : {labels_for_tokens[0][:50]}")
        print(f"Positions Indexes : {positions_indexes[0][:50]}")

        for i in range(num_pdf):

            tokenized_utterances_per_pdf, labels_for_tokens_per_pdf, positions_indexes_per_pdf = [], [], []
            for j in range(len(tokenized_utterances[i])):
                for k in range(len(tokenized_utterances[i][j])):
                    try:
                        positions_indexes_per_pdf.append(positions_indexes[i][j][k])
                        labels_for_tokens_per_pdf.append(labels_for_tokens[i][j][k])
                        tokenized_utterances_per_pdf.append(tokenized_utterances[i][j][k])
                    except:
                        pass

            # Créer une liste de tuples
            dataset_tuple = list(zip(positions_indexes_per_pdf, tokenized_utterances_per_pdf, labels_for_tokens_per_pdf))

            # Trier la liste de tuples par les index de position
            dataset_tuple_sorted = sorted(dataset_tuple, key=lambda x: x[0])

            # Dézipper les tuples triés pour obtenir les listes triées séparément
            dataset_labelled = list(zip(*dataset_tuple_sorted)) # Convertir les tuples en listes

            if len(dataset_labelled) > 0:
                positions.append(list(dataset_labelled[0]))
                tokens.append(list(dataset_labelled[1]))
                token_labels.append(list(dataset_labelled[2]))

        print("\nAfter sorting :")
        print(f"Tokens : {tokens[0][:50]}")
        print(f"Token Labels : {token_labels[0][:50]}")
        print(f"Positions Indexes : {positions[0][:50]}")

        return tokens, token_labels, positions

    def tokenize_and_align_labels(self, data):

        # The given "token_labels" may not match up with the BERT wordpiece tokenization so
        #  this function will map them to the tokenization that BERT uses
        #  -100 is a reserved for labels where we do not want to calculate logs so BERT doesn't waste time
        #  trying to predict tokens like CLS or SEP

        tokenized_inputs = self.tokenizer(data["tokens"], truncation=True, is_split_into_words=True)

        labels = []
        for i, label in enumerate(data[f"token_labels"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                if word_idx is None:  # Set the special tokens to -100.
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)  # CLS and SEP are labeled as -100
                previous_word_idx = word_idx
            labels.append(label_ids)

        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    def preprocess_function(self, data):
        '''Function to preprocess the dataset for sequence classification'''
        # simple function to batch tokenize utterances with truncation
        return self.tokenizer(data["utterance"], truncation=True)

    # SET PRE TRAINED MODEL
    def set_tokenizer(self):
        '''Function to load the pre-trained model for token classification'''
        # On importe le tokenizer de "Model_Name" pre-entrainé
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name) # .encode(), .decode()
        # tokenizer.encode("Madame Pierre DUPONT", truncation=True, padding=True), "labels": [1, 1, 1]})
        prompt = "Monsieur Pierre TANGAMA"
        print("\nExample of Tokenizer : {} became {}".format(prompt, self.tokenizer.encode((prompt))))
        return

    def set_token_classification_model(self):
        '''Function to load the pre-trained model for token classification'''
        # On charge le modèle pré-entrainé de "Model_Name" pour la classification de tokens.
        # self.token_classification_model = AutoModelForCausalLM.from_pretrained(self.model_name)

        print("\nSET PRE-TRAINED TOKEN CLASSIFICATION_MODEL : {}".format(self.model_name))
        self.token_classification_model = AutoModelForTokenClassification.from_pretrained(
            self.model_name,
            num_labels=len(self.unique_token_labels),
            ignore_mismatched_sizes=True,
            #use_auth_token=api_key,
        )
        # On définie les noms de chaque type de token rajouté au modèle
        self.token_classification_model.config.id2label = {
            i: l for i, l in enumerate(self.unique_token_labels)
        }
        #print("ID2Label : {}".format(self.token_classification_model.config.id2label))
        #print("\nToken Classification Model : {}".format(self.token_classification_model))

        # DataCollator crée un lot de données. Il pad également dynamiquement le texte pour qu'il soit de la même longueur que l'élément le plus long du lot, ce qui rend tous les éléments de la même longueur.
        # Il est possible de pad le texte dans la fonction du tokenizer en utilisant padding=True, mais le padding dynamique est plus efficace.
        # L'attention masquée (attention mask) est utilisée pour ignorer les scores d'attention pour les tokens de remplissage.
        self.token_data_collator = DataCollatorForTokenClassification(tokenizer=self.tokenizer)
        return

    def set_sequence_classification_model(self):
        '''Function to load the pre-trained model for token classification'''
        # On charge le modèle pré-entrainé de "Model_Name" pour la classification de séquences.

        print("\nSET PRE-TRAINED SEQUENCE CLASSIFICATION_MODEL : {}".format(self.model_name))
        self.sequence_classification_model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=len(self.unique_sequence_labels),
            ignore_mismatched_sizes=True,
            #use_auth_token=api_key
        )
        # On définie les noms de chaque type d'attribut (séquence) rajouté au modèle
        self.sequence_classification_model.config.id2label = {
            i: l for i, l in enumerate(self.unique_sequence_labels)
        }
        #print("ID2Label : {}".format(self.sequence_classification_model.config.id2label))
        #print("\nSequence Classification Model : {}".format(self.token_classification_model))

        # DataCollatorWithPadding crée un lot de données. Il pad également dynamiquement le texte pour qu'il soit de la même longueur que l'élément le plus long du lot, ce qui rend tous les éléments de la même longueur.
        # Il est possible de pad le texte dans la fonction du tokenizer en utilisant padding=True, mais le padding dynamique est plus efficace.
        # L'attention masquée (attention mask) est utilisée pour ignorer les scores d'attention pour les tokens de remplissage.
        self.sequence_data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)
        return

    # TRAIN & TEST

    def train_token_classification_model(self, epochs=1, batch_size=1):
        '''Function to train the model'''

        # epochs = Number of checkpoints
        # batch_size = Size of each batch

        # path_output_dir = os.path.join(os.path.dirname(__file__), '..', 'model', 'tok_clf_model')
        # path_logging_dir = os.path.join(os.path.dirname(__file__), '..', 'model', 'logs.json')
        path_output_dir = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model/tok_clf_model'
        #path_logging_dir = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model'

        training_args = TrainingArguments(
            output_dir=path_output_dir, # Location to save snapshot of checkpoint
            logging_dir=path_output_dir+'/logs',
            num_train_epochs=epochs, # Smaller number of epochs for testing = Smaller time of training
            per_device_train_batch_size=batch_size, # Size of each batch must be around 32 or 64
            per_device_eval_batch_size=batch_size, # Smaller batch size for testing = More chances to update the model
            load_best_model_at_end=True, # To ignore overtraining and only keep the best model

            logging_steps=1, # Number of step to output the log of the step
            log_level='info', # Level of details at highest 'info'
            logging_strategy='steps',
            eval_strategy='epoch',
            save_strategy='epoch'
        )

        # Define the loss logger

        #loss_logger = LossLogger()
        #callbacks=[loss_logger]

        # Define the trainer:

        trainer = CustomTrainer(
            model=self.token_classification_model,
            args=training_args,
            train_dataset=self.token_classification_dataset['train'],
            eval_dataset=self.token_classification_dataset['test'],
            compute_metrics=self.compute_metrics_precision,
            #compute_metrics=self.compute_metrics,
            data_collator=self.token_data_collator,
        )

        # Evaluate and train the model

        print("\nTRAIN TOKEN CLASSIFICATION MODEL :\n")
        #for epoch in tqdm(range(epochs), desc="Evaluation before training...", unit="epoch", dynamic_ncols=True):
        #    trainer.evaluate()

        #for epoch in tqdm(range(epochs), desc="Training the model..."):
        #    trainer.train()

        #for epoch in tqdm(range(epochs), desc="Evaluation after training..."):
        #    trainer.evaluate()

        print("\nEvaluation before training...\n")
        print(trainer.evaluate())
        print("\nTraining the model...\n")
        print(trainer.train())
        print("\nEvaluation after training ...\n")
        print(trainer.evaluate())

        self.logs['Token Classification Model'] = trainer.get_loss_logs()

        trainer.save_model()

        return

    def train_sequence_classification_model(self, epochs=1, batch_size=32):
        '''Function to train the model'''

        # epochs = Number of checkpoints
        # batch_size = Size of each batch

        # path_output_dir = os.path.join(os.path.dirname(__file__), '..', 'model', 'seq_clf_model')
        # path_logging_dir = os.path.join(os.path.dirname(__file__), '..', 'model', 'logs.json')
        path_output_dir = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model/seq_clf_model'
        #path_logging_dir = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model' #/logs.json'

        training_args = TrainingArguments(
            output_dir=path_output_dir, # Location to save snapshot of checkpoint
            logging_dir=path_output_dir+'/logs',
            num_train_epochs=epochs, # Smaller number of epochs for testing = Smaller time of training
            per_device_train_batch_size=batch_size, # Size of each batch must be around 32 or 64
            per_device_eval_batch_size=batch_size, # Smaller batch size for testing = More chances to update the model
            load_best_model_at_end=True, # To ignore overtraining and only keep the best model

            logging_steps=10, # Number of step to output the log of the step
            log_level='info', # Level of details at highest 'info' #log_level=logging.ERROR pour minimiser le temps d'execution
            logging_strategy='steps',
            eval_strategy='epoch', #'steps' other possibility
            save_strategy='epoch'
        )

        #Hyper Parameters must be optimised
        # some deep learning parameters that the Trainer is able to take in
        warmup_steps=len(self.sequence_classification_dataset['train']) // 5,  # number of warmup steps for learning rate scheduler,
        weight_decay = 0.05,


        # Define the trainer:

        trainer = CustomTrainer(
            model=self.sequence_classification_model,
            args=training_args,
            train_dataset=self.sequence_classification_dataset['train'],
            eval_dataset=self.sequence_classification_dataset['test'],
            compute_metrics=self.compute_metrics_precision,
            #compute_metrics=self.compute_metrics
            data_collator=self.sequence_data_collator
        )

        # Evaluate and train the model
        #print("\nTRAIN SEQUENCE CLASSIFICATION MODEL :\n")
        #for epoch in tqdm(range(epochs), desc="Evaluation before training..."):
        #    trainer.evaluate()
        #
        #for epoch in tqdm(range(epochs), desc="Training the model..."):
        #    trainer.train()
        #
        #for epoch in tqdm(range(epochs), desc="Evaluation after training..."):
        #    trainer.evaluate()

        print("\nTRAIN SEQUENCE CLASSIFICATION MODEL :\n")
        print("\nEvaluation before training\n")
        print(trainer.evaluate())
        print("\nTraining the model\n")
        print(trainer.train())
        print("\nEvaluation after training\n")
        print(trainer.evaluate())

        self.logs['Sequence Classification Model'] = trainer.get_loss_logs()
        trainer.save_model()

        return

    def test_token_classification_model(self, example):
        '''Function to test the model'''
        pipe = pipeline(
            'token-classification',
            model=self.token_classification_model,
            tokenizer=self.tokenizer,
        )
        return pipe(example)

    def test_sequence_classification_model(self, example):
        '''Function to test the model'''
        pipe = pipeline(
            'text-classification',
            model=self.sequence_classification_model, # TODO : Enlever le "model ="
            tokenizer=self.tokenizer,
        )
        return pipe(example)

        # If model saved in a directory ./snips_seq_clf.results
        ''' pipe = pipeline(
                'text-classification',
                './snips_seq_clf/results',
                tokenizer=self.tokenizer
            )
        '''

    # UTILS

    def plot_loss_function(self, type_display="loss"):
        '''Fonction pour tracer la fonction de coût / l'évolution du taux d'apprentissage / l'évolution de la norme du gradient'''

        #type_display = "loss" / "learning_rate" / "grad_norm"
        plt.figure(figsize=(10, 6))  # Ajuster la taille de la figure pour une meilleure visualisation
        name_function = {"loss": "Fonction Coût", "learning_rate": "Taux d'apprentissage", "grad_norm": "Norme du gradient"}
        plt.title(f"Evolution de la {name_function[type_display]} ({type_display} function)", fontsize=14)  # Augmenter la taille de la police

        # Tracer de la première courbe
        #name1 = "Token Classification Model"
        #ax1.plot(self.logs[name1][type_display], color='blue', label=name1)
        #ax1.set_xlabel("Itération", fontsize=12)  # Augmenter la taille de la police

        #name2 = "Sequence Classification Model"
        #ax2.plot(self.logs[name2][type_display], color='red', label=name2)
        #ax2.set_xlabel("Itération", fontsize=12)  # Augmenter la taille de la police

        for name, log in self.logs.items():
            plt.plot(log["epoch"][:len(log[type_display])], log[type_display], label=name)

        # Ajout d'une courbe lissée
        window_size = 10
        epochSmooth = np.convolve(logs["Token Classification Model"][type_display], np.ones(window_size)/window_size, mode='valid')
        xSmooth = np.arange(len(epochSmooth))
        xSmooth_normalized = np.max(logs["Token Classification Model"]["epoch"])*(xSmooth - np.min(xSmooth)) / (np.max(xSmooth) - np.min(xSmooth))
        plt.plot(xSmooth_normalized, epochSmooth, label=f"Smooth Token Classification Model : x{window_size}")

        plt.xlabel("Epoch", fontsize=12)  # Augmenter la taille de la police
        plt.ylabel(f"{name_function[type_display]}", fontsize=12)
        plt.legend()  # Ajouter une légende pour distinguer les différentes courbes de coût
        plt.grid(True)  # Ajouter une grille pour une meilleure lisibilité
        plt.show()

    def get_loss_logs(self):
        '''Function to get the loss logs'''
        return self.logs

    def load_loss(self, clean_loss=False):
        '''Function to load the logs'''
        json_loss = {}
        try:
            if (clean_loss):
                with open(self.path_loss, 'w') as json_file:
                    json.dump({}, json_file)
                    print("Le fichier Loss JSON situé à l'emplacement {} a été nettoyé.".format(self.path_loss))
            else:
                print("Import Recorded Losses...")
                with open(self.path_loss, 'r') as json_file:
                    json_loss = json.load(json_file)
        except Exception as e:
            print("Une erreur est survenue:", str(e))
        else:
            print("Création Loss JSON file au :", self.path_loss)
            json_loss["Type"] = "Attestation de propriété"
            json_loss["DateTime"] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        index = len(json_loss.keys()) - 1
        json_loss["Loss N°" + str(index)] = self.logs

        with open(self.path_loss, 'w') as json_file:
            json.dump(json_loss, json_file, indent=4)

        return json_loss

    def freeze(model, n):
        '''Function to freeze n encoder layers'''

        #BERT MEDIUM = 12 encoders
        #BERT LARGE = 24 encoders
        #More param freezed = More speed = Less Accuracy

        ref_name = 'encoder.layer.' + str(n)
        for name, param in model.distilbert.parameters():
            if ref_name in name:
                break
            param.requires_grad = False  # disable training in BERT
        return model.distilbert.parameters

    def compute_metrics(eval_pred):
        '''Function to compute the metrics of the model'''
        # PAS DE SELF ICI
        # custom method to take in logits and calculate accuracy of the eval set
        # logits is a matrix of logits where the number of rows is the batch size and the number of columns is the number of sequences labels
        # labels are the true labels as a reference of correctly predicted labels
        # predictions are the predicted labels
        #metric = load("accuracy") ### "accuracy" / "precision", "squad", average="weighted", "macro", "micro", "samples", "binary"
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels) #, sample_weight=[0.5, 2, 0.7, 0.5, 9, 0.4]

    def compute_metrics_precision(logits, labels):
        '''Function to compute the metrics of the model'''
        #metric = load_metric("accuracy")
        logits, labels = logits, labels
        predictions = np.argmax(logits, axis=-1)
        precision = precision_score(labels, predictions, average='weighted')  # 'weighted' to account for label imbalance
        #return metric.compute(predictions=predictions, references=labels) #, sample_weight=[0.5, 2, 0.7, 0.5, 9, 0.4]
        return {"precision": precision * 100}

    def get_parameters(self):
        parameters = [self.token_classification_model.distilbert.parameters, self.sequence_classification_model.distilbert.parameters]
        print("Parameters Token Classification : {}".format(parameters[0]))
        print("Parameters Sequence Classification : {}".format(parameters[1]))
        return parameters

    def get_config_model(self):
        '''Function to get the config of the model'''
        config = AutoConfig.from_pretrained(self.model_name)
        print("Configuration Model : {}".format(config))

    def generated_text(self, prompt):
        generator = pipeline('text-generation', model=self.token_classification_model, tokenizer=self.tokenizer)
        return generator(prompt, max_length=100)

# Class Trainer personnalisée pour stocker les valeurs de loss
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_logs = {}
        self.loss_logs['loss'] = []
        self.loss_logs['learning_rate'] = []
        self.loss_logs['grad_norm'] = []
        self.loss_logs['epoch'] = []
        self.loss_logs['precision'] = []

    def log(self, logs):
        super().log(logs)
        if logs.get('precision'):
            self.loss_logs['precision'].append(logs['precision'])
        if logs.get('loss'):
            self.loss_logs['loss'].append(logs['loss'])
        if logs.get('learning_rate'):
            self.loss_logs['learning_rate'].append(logs['learning_rate'])
        if logs.get('grad_norm'):
            self.loss_logs['grad_norm'].append(logs['grad_norm'])
        if logs.get('epoch'):
            self.loss_logs['epoch'].append(logs['epoch'])

    def get_loss_logs(self):
        return self.loss_logs

class LossLogger(TrainerCallback):

    def on_train_begin(self, args, state, control):
        self.logs = []

    def on_step_end(self, args, state, control):
        self.logs.append(state.log_history["loss"][-1])


    def on_train_epoch_end(self, args, state, control):
        print(f"Loss : {state.log_history[-1]['loss']}")

    #def on_train_end(self, args, state, control, **kwargs):
    #    print(f"Loss : {state.log_history[-1]['loss']}")

    #def on_epoch_end(self, args, state, control, **kwargs):
    #    print(f"Loss : {state.log_history[-1]['loss']}")

if __name__ == "__main__":

    json_name = "attestation_propriete_train_dataset.json"

    classification_model = Classification_Model()

    # SET TOKENIZER
    classification_model.set_tokenizer()

    # SET DATASET
    classification_model.load_dataset(json_name=json_name)
    classification_model.set_train_dataset()

    # TOKEN CLASSIFICATION MODEL
    classification_model.set_token_classification_model()
    classification_model.train_token_classification_model()

    # TEST TOKEN CLASSIFICATION
    example = "Monsieur Pierre TANGAMA"
    print("\nTEST TOKEN CLASSIFICATION WITH EXAMPLE : {}".format(example))
    results_tok_clf_model = classification_model.test_token_classification_model(example)#, path_model=path_sequence_classification_trained_model)
    print(results_tok_clf_model)
    for result in results_tok_clf_model:
        print(f"Token: {result['word']}, Label: {result['entity']}, Score: {result['score']}")

    # SEQUENCE CLASSIFICATION MODEL
    classification_model.set_sequence_classification_model()
    classification_model.train_sequence_classification_model()

    # TEST SEQUENCE CLASSIFICATION
    print("\nTEST SEQUENCE CLASSIFICATION WITH EXAMPLE : {}".format(example))
    results_seq_clf_model = classification_model.test_sequence_classification_model(example)#, path_model=path_token_classification_trained_model)
    print(results_seq_clf_model)

    # UTILS
    #classification_model.freeze(classification_model.token_classification_model, 12)
    #classification_model.get_parameters()
    #classification_model.load_loss()
    #classification_model.plot_loss_function()
    #classification_model.get_config_model()

    #prompt = "Salut Mistral ! Peux-tu m'expliquer ce que consiste à fine-tuner un modèle pré-entrainé d'intelligence artificielle ?"
    #print("\nDiscussion avec le modèle :")
    #print("\nPrompt : {}".format(prompt))
    #print(classification_model.generated_text(prompt)[0]['generated_text'])

    #classification_model.get_sequence_classification_tokenized_dataset()
    #classification_model.get_token_classification_tokenized_dataset()

    # TODO : Compute Metrics (One for Token Classification / Text Classification in Evaluate - HuggingFace)
    # TODO : Callbacks to print loss during training
    # TODO : Test the model on Mistral


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/6ea81172465e8b0ad3fddeed32b986cdcdcffcf0/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.42.4",
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/6ea81172465e8b0ad3fddeed32b986cdcdcffcf0/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-ba


Example of Tokenizer : Monsieur Pierre TANGAMA became [101, 25599, 4855, 157, 14962, 10583, 8271, 102]

Set Dataset for Sequence and Token Classification

Example of the dataset :
PDF n°0 / Sequence example : 150
Sequence label :  Context / Total Length Per Pdf :  2525
Utterance :  notamment / Total Length Per Pdf :  2525
Tokens :  ['notamment'] / Total Length Per Pdf :  2525
Token labels :  ['O'] / Total Length Per Pdf :  2525
Positions Indexes :  [880] / Total Length Per Pdf :  2525

There are 70 unique sequence labels :
['Certification_Attestation_S', 'Nom_Societe_1', 'Eval_Quotites_Transmises_5', 'Valeur_Transmis_8', 'Date_Naissance_D_3', 'Situation_Maritale_H_3', 'Regime_Propriete_6', 'Designation_S', 'Profession_D_3', 'Type_Document_2', 'Adresse_Propriete_4', 'Prefixe_TC_4', 'Servitude_S', 'Nom_D_3', 'Nationalite_H_3', 'Qualite_Resident_D_3', 'Contribution_Securite_Immobiliere_9', 'Evaluation_S', 'Lieu_Naissance_D_33', 'Certification_Identite_S', 'Statut_Publication_7', 'Lieu_Na

Map:   0%|          | 0/147 [00:00<?, ? examples/s]


Token Classification Dataset after alignment : PDF n°0
[46, 46, 21, 1, 1, 1, 1, 1, 59, 73, 35, 35, 35, 35, 35, 35, 35, 35, 14, 35, 69, 35, 47, 35, 54, 35, 24, 35, 24, 35, 24, 35, 24, 35, 74, 35, 35, 35, 35, 35, 76, 71, 35, 35, 71, 35, 71, 35, 71, 71, 35, 35, 71, 35, 57, 35, 35, 38, 35, 0, 35, 0, 35, 0, 35, 2, 35, 11, 35, 97, 35, 22, 35, 35, 35, 35, 101, 88, 38, 93, 0, 109, 0, 109, 0, 109, 0, 96, 78, 2, 11, 64, 10, 64, 18, 10, 97, 22, 46, 14, 15, 69, 46, 47, 46, 46, 88, 46, 12, 12, 46, 46, 46, 46, 46, 61, 34, 46, 46, 12, 46, 46, 46, 12, 46, 46, 46, 46, 12, 46, 46, 12, 46, 46, 12, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 12, 46, 12, 46, 46, 46, 46, 46, 12, 46, 12, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 12, 46, 46, 46, 46, 46, 46, 12, 46, 46, 12, 46, 46, 46, 46, 12, 46, 46, 46, 46, 46, 46, 12, 46, 46, 46, 42, 46, 93, 46, 46, 46, 46, 109, 109, 46, 46, 46, 61, 14, 34, 69, 23, 47, 70, 49, 65, 7, 109, 46, 109, 38, 2, 11, 97, 22, 109, 88, 12, 109, 12, 42, 38, 2, 11, 97

Map:   0%|          | 0/316375 [00:00<?, ? examples/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/6ea81172465e8b0ad3fddeed32b986cdcdcffcf0/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",


Sequence Classification Dataset after alignment : Sequence n°150
{'utterance': 'ci-après', 'label': 63, 'tokens': ['ci-après'], 'token_labels': [46], 'input_ids': [101, 172, 1182, 118, 170, 1643, 1197, 10695, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


Sequence Classification Tokenized Snips Cleaned Formatted :
DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 253100
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 63275
    })
})

Dataset In Place !

SET PRE-TRAINED TOKEN CLASSIFICATION_MODEL : distilbert-base-cased


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas


TRAIN TOKEN CLASSIFICATION MODEL :


Evaluation before training...



InvalidParameterError: The 'y_true' parameter of precision_score must be an array-like or a sparse matrix. Got <transformers.trainer_utils.EvalPrediction object at 0x7c3773fd6bc0> instead.

### Analysis Model

In [ ]:
import numpy
# UTILS
#classification_model.freeze(classification_model.token_classification_model, 12)
#classification_model.get_parameters()
#classification_model.get_config_model()
#classification_model.load_loss()
#classification_model.plot_loss_function(type_display="loss") # loss / learning_rate / grad_norm

def plot_loss_function_2(logs, type_display="loss"):
        '''Fonction pour tracer la fonction de coût / l'évolution du taux d'apprentissage / l'évolution de la norme du gradient'''

        #type_display = "loss" / "learning_rate" / "grad_norm"
        plt.figure(figsize=(10, 6))  # Ajuster la taille de la figure pour une meilleure visualisation
        name_function = {"loss": "Fonction Coût", "learning_rate": "Taux d'apprentissage", "grad_norm": "Norme du gradient"}
        plt.title(f"Evolution de la {name_function[type_display]} ({type_display} function)", fontsize=14)  # Augmenter la taille de la police

        for name, log in logs.items():
            plt.plot(log["epoch"][:len(log[type_display])], log[type_display], label=name)

        # Ajout d'une courbe lissée
        window_size = 10
        epochSmooth = np.convolve(logs["Token Classification Model"][type_display], np.ones(window_size)/window_size, mode='valid')
        xSmooth = np.arange(len(epochSmooth))
        xSmooth_normalized = np.max(logs["Token Classification Model"]["epoch"])*(xSmooth - np.min(xSmooth)) / (np.max(xSmooth) - np.min(xSmooth))
        plt.plot(xSmooth_normalized, epochSmooth, label=f"Smooth Token Classification Model : x{window_size}")

        plt.xlabel("Epoch", fontsize=12)  # Augmenter la taille de la police
        plt.ylabel(f"{name_function[type_display]}", fontsize=12)
        plt.legend()  # Ajouter une légende pour distinguer les différentes courbes de coût
        plt.grid(True)  # Ajouter une grille pour une meilleure lisibilité
        plt.show()


# "precision" = Précision = Vrais Positifs / (Vrais Positifs + Faux Positifs)
# "accuracy" = Exactitude = (Vrais Positifs + Vrais Négatifs) / (Vrais Positifs + Faux Positifs + Vrais Négatifs + Faux Négatifs)
print(classification_model.logs)
logs = classification_model.get_loss_logs()
#plot_loss_function_2(logs, type_display="grad_norm") # alpha = -5e-5
#plot_loss_function_2(logs, type_display="loss")
plot_loss_function_2(logs, type_display="precision")

### Inference Model

In [ ]:
import os
import json
import pdfplumber
import pathlib
import pprint
from transformers import pipeline, AutoTokenizer

class Formalist_IA_Model():
    #https://www.youtube.com/watch?v=tiZFewofSLM
    def __init__(self, path_token_classification_trained_model, path_sequence_classification_trained_model, model_name="distilbert-base-cased"):
        #self.path_tok_clf_model = os.path.join(os.path.dirname(__file__), '..', 'model', 'tok_clf_model')
        #self.path_seq_clf_model = os.path.join(os.path.dirname(__file__), '..', 'model', 'seq_clf_model')
        #self.path_pdf_input = os.path.join(os.path.dirname(__file__), '..', 'pdf_input')
        #self.path_json_output = os.path.join(os.path.dirname(__file__), '..', 'json_output')
        self.path_tok_clf_model = path_token_classification_trained_model
        self.path_seq_clf_model = path_sequence_classification_trained_model
        #self.path_pdf_input = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model'
        self.path_json_output = '/content/drive/My Drive/IA/IA Notaire - BellVision/Model'
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.output_json_name = ""
        self.number_of_pdf = ""
        self.number_of_pages_per_pdf = []
        self.texts_per_pdf = []

    def load_pdf(self, path_pdf_input='/content/drive/My Drive/IA/IA Notaire - BellVision/Data/Depot_PDF'):
        '''Chargement pdf'''
        paths_pdf = pathlib.Path(path_pdf_input).rglob('*.pdf')
        paths_pdf = [str(path_pdf) for path_pdf in paths_pdf]
        self.names_pdf = [os.path.basename(path_pdf) for path_pdf in paths_pdf]
        self.number_of_pdf = len(paths_pdf)

        for path_pdf in paths_pdf:
            with pdfplumber.open(path_pdf) as pdf:
                pages = pdf.pages
                self.number_of_pages_per_pdf.append(len(pages))
                self.texts_per_pdf.append([pages[i].extract_text() for i in range(self.number_of_pages_per_pdf[-1])])
                #self.lines = [self.texts[i].split("\num_match") for i in range(self.num_pages)]

    def read_pdf(self):
        '''Lecture pdf'''
        isDisplayed = False
        for i, texts in enumerate(self.texts_per_pdf):
            print("\nLecture du PDF {} en cours".format(self.names_pdf[i]))
            for num_page, page in enumerate(texts):
                if not isDisplayed:
                    print("\nNumero de Page : {}".format(num_page + 1))
                    print("\nPage : {}".format(page))
                    isDisplayed = True

    def test_token_classification_model(self, example):
        '''Function to test the model'''
        pipe = pipeline(
            'token-classification', #"ner"
            self.path_tok_clf_model,
            tokenizer=self.tokenizer,
            #use_auth_token=api_key
        )
        return pipe(example) # Possibility to passed several inputs pipe(examples) avec examples = list([example1, ..., exemple_n])

    def test_sequence_classification_model(self, example):
        '''Function to test the model'''
        pipe = pipeline(
            'text-classification',  #"sequence-classification", #"zero-shot-classification",
            self.path_seq_clf_model,
            tokenizer=self.tokenizer,
            #truncation = true
            #use_auth_token=api_key
        )
        return pipe(example)

    def token_classifier(self):
        '''Fonction de classification en token'''

        token_results = []
        for i, texts in enumerate(self.texts_per_pdf):
            print("\nLecture du PDF {} en cours".format(self.names_pdf[i]))
            token_results_per_pdf = []
            for num_page, page in enumerate(texts):
                #print("\nNumero de Page : {}\n".format(num_page + 1))
                #print("\nPage : {}\n".format(page))
                token_results_per_pdf.append(self.test_token_classification_model(page.replace("\n", " ")))
            token_results.append(token_results_per_pdf)

        return token_results

    def sequence_classifier(self):
        '''Fonction de classification en séquence'''
        return

    def get_possible_sequences():
        '''Fonction pour récupérer les résultats possibles'''
        return

    def filter_best_sequences(self):
        '''Fonction pour récupérer la séquence la plus probable'''
        return

    def set_tokenizer(self):
        '''Fonction pour définir le tokenizer'''
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def validate(self):
        '''Fonction pour valider le modèle'''
        return

    def export_to_json_output(self, path_json_output, output_json_name):
        '''Fonction pour exporter les résultats'''
        return


if __name__ == "__main__":

    #path_token_classification_trained_model = ".\\snips_tok_clf\\results"
    #path_sequence_classification_trained_model = ".\\snips_seq_clf\\results"
    path_token_classification_trained_model = "/content/drive/My Drive/IA/IA Notaire - BellVision/Model/tok_clf_model"
    path_sequence_classification_trained_model = "/content/drive/My Drive/IA/IA Notaire - BellVision/Model/seq_clf_model"

    formalist_ia_model = Formalist_IA_Model(path_token_classification_trained_model, path_sequence_classification_trained_model)

    #path_data_folder = os.path.join(os.path.dirname(__file__), '..', 'data')
    path_data_folder = '/content/drive/My Drive/IA/IA Notaire - BellVision/Data/Depot_PDF'


    formalist_ia_model.load_pdf(path_data_folder)
    formalist_ia_model.read_pdf()

    token_classification_results = formalist_ia_model.token_classifier()

    for results_per_pdf in token_classification_results:
        for results_per_page in results_per_pdf:
            for result in results_per_page:
                print(f"Token: {result['word']}, Label: {result['entity']}, Score: {result['score']}")

    sequence_classifcation_results = formalist_ia_model.sequence_classifier()

    output_json_name = "analysis_report.json"
    path_json_output = os.path.join(os.path.dirname(__file__), '..', 'json_output')

    formalist_ia_model.export_to_json_output(path_json_output, output_json_name)





    #example = "Monsieur Pierre TANGAMA"
    #print("\nTEST TOKEN CLASSIFICATION MODEL : {}\n".format(example))
    #results = formalist_ia_model.test_token_classification_model(example)
    #for result in results:
    #    print(f"Token: {result['word']}, Label: {result['entity']}, Score: {result['score']}")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/6ea81172465e8b0ad3fddeed32b986cdcdcffcf0/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.41.2",
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/6ea81172465e8b0ad3fddeed32b986cdcdcffcf0/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-ba


Lecture du PDF API TANGAMA.pdf en cours

Numero de Page : 1

Page : 100556503
CB/PT/
L'AN DE U X MILLE VINGT TROIS,
LE V I N G T FÉVRIER
A LE TAMPON (Réunion), 106, rue Jules Bertaut, au siège de la Société
Civile Professionnelle ci-après nommée,
Maître Catherine BRAUGE, soussignée, Notaire, au sein de la société
dénommée « Jean-Patrick MOUTIEN, Corinne ROSSOLIN et Rodolphe BEHAR,
notaires, associés d'une société civile professionnelle titulaire d'un office
notarial », sise LE TAMPON (97430), 106, rue Jules Bertaut.
A reçu le présent acte contenant ATTESTATION IMMOBILIERE APRES
DECES AVEC CONSTATATION DE RETOUR CONVENTIONNEL à la requête du
DONATEUR :
- Madame Amélie VIRY présente à l’acte.
Ci-après nommée, domiciliée et qualifiée.
TEXTE APPLICABLE
L’article 29 du décret numéro 55-22 du 4 janvier 1955 dispose notamment que
toute transmission ou constitution par décès de droits réels immobiliers doit être
constatée par une attestation notariée.
ATTENDU
I - Le décès et la dévolution suc

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
    "98": "E-Prefixe_Profession",
    "99": "B-Euro",
    "100": "B-Prefixe_Nationalite",
    "101": "E-Nationalite",
    "102": "B-Relation_Familiale",
    "103": "I-Part`_En_Chiffre",
    "104": "I-Surface",
    "105": "E-Statut_Publication",
    "106": "B-Numero",
    "107": "B-Volume_Ref",
    "108": "B-Ville",
    "109": "E-Prefixe_Part",
    "110": "E-Cert_Id_11"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_100": 100,
    "LABEL_101": 101,
    "LABEL_102": 102,
    "LABEL_103": 103,
    "LABEL_104": 104,
    "LABEL_105": 105,
    "LABEL_106": 106,
    "LABEL_107": 107,
    "LABEL_108": 108,
    "LABEL_109": 109,
    "LABEL_11": 11,
    "LABEL_110": 110,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LABEL_16": 16,
    "LABEL_17": 17,
    "LABEL_18": 18,
    "LABEL_19": 19,
    "LABEL_2": 2,
    

Token: L, Label: B-Annee_Lettre, Score: 0.483190655708313
Token: ', Label: I-Annee_Lettre, Score: 0.30970701575279236
Token: AN, Label: I-Annee_Lettre, Score: 0.7743973135948181
Token: DE, Label: I-Annee_Lettre, Score: 0.924241304397583
Token: U, Label: I-Annee_Lettre, Score: 0.9247045516967773
Token: X, Label: I-Annee_Lettre, Score: 0.935035228729248
Token: MI, Label: I-Annee_Lettre, Score: 0.9354400038719177
Token: ##LL, Label: I-Annee_Lettre, Score: 0.9109214544296265
Token: ##E, Label: I-Annee_Lettre, Score: 0.9081212282180786
Token: VI, Label: I-Annee_Lettre, Score: 0.9069433808326721
Token: ##NG, Label: I-Annee_Lettre, Score: 0.6974400281906128
Token: ##T, Label: I-Annee_Lettre, Score: 0.46344348788261414
Token: T, Label: I-Date_Lettre, Score: 0.19631031155586243
Token: ##RO, Label: I-Date_Lettre, Score: 0.23320454359054565
Token: ##IS, Label: I-Date_Lettre, Score: 0.25002574920654297
Token: ,, Label: I-Date_Lettre, Score: 0.08852735906839371
Token: L, Label: B-Date_Lettre, Score

TypeError: 'NoneType' object is not iterable